In [220]:
#!pip install ffmpeg-python>=0.2.0
#!pip install matplotlib>=3.0.2
#!pip install munkres>=1.1.2
#!pip install numpy>=1.16
#!pip install opencv-python>=3.4
#!pip install Pillow>=5.4
#!pip install vidgear>=0.1.4
#!pip install torch>=1.4.0
#!pip install torchvision>=0.5.0
#!pip install tqdm>=4.26
#!pip install tensorboard>=1.11
#!pip install tensorboardX>=1.4
#!pip install backports.entry-points-selectable
#!pip install backports.tempfile
#!pip uninstall -y setuptools
#!pip install setuptools==65.5.0
#!pip install fastdtw

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Imports

In [221]:
# Essential Imports
import cv2
import numpy as np
import os
import time
import torch
import pathlib
import math
import re
import pandas as pd
import matplotlib.pyplot as plt
from SimpleHRNet import SimpleHRNet
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.signal import find_peaks
import scipy.ndimage

### Constants & Configuration

**Purpose:**

This cell defines all the essential constants, file paths, processing parameters, and exercise-specific configurations used throughout the notebook. It centralizes settings, making it easier to modify the script's behavior without searching through multiple functions.

* Paths determine data sources and model location.
* Joint settings ensure consistency with the pose model.
* Processing parameters directly influence normalization, matching sensitivity (thresholds), and repetition counting accuracy (peak detection, smoothing). The parameters grouped under "User Settings & Tuning Parameters" are the ones most likely needing adjustment to optimize performance for your specific videos and exercises.
* `EXERCISE_CONFIG` dictates the core signal processing logic used to analyze the movement pattern for repetition analysis for each distinct exercise type.

**Outcomes:**

Running this cell does not return any values directly but defines the following variables in the notebook's memory, making them available to subsequent cells:

* **File/Model Paths (User Must Set):**
    * `MODEL_PATH`: ( `pathlib.Path`) - The full path to the downloaded SimpleHRNet model weights file (e.g., `.pth`).
    * `BASELINE_ROOT_DIR`: ( `pathlib.Path`) - The full path to the main folder containing subfolders for each baseline exercise video (ensure filenames include angle and rep count, e.g., `squat_10reps_front_video1.mp4`).
    * `VIDEO_TO_RECOGNIZE_PATH`: ( `pathlib.Path`) - The full path to the default test video file you want to analyze later.
* **Model & Pose Structure:**
    * `EXPECTED_JOINT_COUNT`: ( `int`, Unit = Count) - The number of keypoints the HRNet model detects (17 for COCO). Should match the loaded model.
    * `JOINT_NAMES_LIST`: ( `list` of `str`, Unit = Name) - Ordered list of names corresponding to the keypoint indices output by the model. Order **must** match the model's output.
* **Core Processing Settings:**
* `TARGET_REP_LENGTH`: ( `int`, Unit = Frames / Timesteps)
        * **Purpose:** This parameter defines a **standard length** (in terms of number of data points or "frames") for every single repetition segment after it has been detected. Real exercise repetitions naturally vary in duration (some reps are faster, some slower). To compare them fairly using DTW or to average them meaningfully for creating a baseline, they need to be represented over the same timescale.
        * **Mechanism:** The `time_normalize_sequence` function takes a repetition segment (which might have, say, 30 frames or 50 frames) and resamples it using linear interpolation to produce a new sequence that always has exactly `TARGET_REP_LENGTH` frames (e.g., 100). This stretches or compresses the time dimension of the repetition.
        * **Effect on DTW Comparison & Sensitivity:** DTW calculates a distance based on the alignment between two sequences. The absolute magnitude of this distance is influenced by the length of the sequences being compared. By normalizing all repetitions (both from baselines and the test video) to the *same target length*, we ensure that the DTW distances primarily reflect differences in the *shape* of the movement over that standard duration, rather than being heavily skewed just because one raw repetition took more frames than another. This standardization makes the `DTW_DISTANCE_THRESHOLD` more consistently applicable across different reps and exercises. Choosing the value (e.g., 100) is often a balance – long enough to capture the details of the movement, but not so long that it excessively increases computation time or overemphasizes noise.
* **User Settings & Tuning Parameters (Require Experimentation):**
    * `DTW_DISTANCE_THRESHOLD`: ( `float`, Unit = Unitless Distance) - **Fallback** threshold for DTW comparison. Used if a specific derived threshold isn't available. Lower values are stricter for matching. *Tuning recommended based on derived threshold outputs (Cell 6.5).*
    * `TUNING_PROMINENCE_RANGE`: ( `list` of `float`, Unit = Unitless Ratio) - Defines the search space for the `peak_prominence` parameter during automatic tuning of repetition counting. Controls how much a peak must stand out relative to its surroundings (0.0 to 1.0). *Tuning the range might be needed.*
    * `TUNING_DISTANCE_RANGE`: ( `list` of `int`, Unit = Frames) - Defines the search space for the `peak_distance` parameter during automatic tuning. Controls the minimum number of frames separating detected repetitions. *Tuning the range might be needed.*
    * `DEFAULT_SMOOTHING_WINDOW`: ( `int`, Unit = Frames) - Size of the moving average window used to smooth the trajectory signal *before* peak detection during tuning and recognition. Helps reduce noise but can flatten small peaks. *Tuning recommended.*
    * `DEFAULT_PEAK_PROMINENCE`: ( `float`, Unit = Unitless Ratio) - **Fallback** prominence value used for repetition counting if tuning fails or isn't performed for an exercise/angle. *Tuning recommended.*
    * `DEFAULT_PEAK_DISTANCE`: ( `int`, Unit = Frames) - **Fallback** minimum distance between peaks used for repetition counting if tuning fails or isn't performed. *Tuning recommended.*
    * `DEFAULT_THRESHOLD_STD_MULTIPLIER`: ( `float`, Unit = Unitless) - Factor (N) used when deriving thresholds automatically (Mean + N * StdDev). Higher values create larger, more lenient thresholds. *Tuning recommended.*
    * `DEFAULT_MIN_REPS_FOR_THRESHOLD`: ( `int`, Unit = Count) - Minimum number of valid repetitions needed within a specific baseline angle category to attempt deriving its threshold.
* **Exercise Configuration:**
    * `EXERCISE_CONFIG`: ( `dict`) - A nested dictionary defining the specific logic for calculating the 1D trajectory used in segmentation/counting for each exercise.
        * `metric`: ( `str`) Specifies the method (e.g., `'elbow_angle'`, `'hip_y'`).
        * `joints`: ( `list` of `str`) Base names of joints needed for the metric.
        * `invert_for_valley`: ( `bool`) Whether the lowest point (`True`) or highest point (`False`) of the metric signal typically marks the end/start of a repetition cycle.

In [222]:
MODEL_PATH = pathlib.Path("C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/simple-HRNet/weights/pose_hrnet_w48_256x192.pth") # Path to HRNet weights
BASELINE_ROOT_DIR = "C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/baselines2"
VIDEO_TO_RECOGNIZE_PATH = "C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/test_videos/test_excercise.mp4" # Video file to analyze

# Model & Pose Settings
EXPECTED_JOINT_COUNT = 17 # COCO keypoints format
JOINT_NAMES_LIST = ["Nose","Left Eye","Right Eye","Left Ear","Right Ear","Left Shoulder",
                    "Right Shoulder","Left Elbow","Right Elbow","Left Wrist","Right Wrist",
                    "Left Hip","Right Hip","Left Knee","Right Knee","Left Ankle","Right Ankle"]

# Recognition & Processing Parameters
TARGET_REP_LENGTH = 100
DTW_DISTANCE_THRESHOLD = 500.0

# Parameters for Repetition Counting Tuning (Grid Search Ranges)
TUNING_PROMINENCE_RANGE = [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14]
TUNING_DISTANCE_RANGE = [5, 8, 10, 12, 15, 20, 25, 30, 35]

# Parameters for Threshold Derivation
DEFAULT_THRESHOLD_STD_MULTIPLIER = 2.0 # Default for mean + N*stddev calculation
DEFAULT_MIN_REPS_FOR_THRESHOLD = 3     # Min reps needed in baseline angle category to derive threshold

# Fallback if tuning fails or not enough data
DEFAULT_SMOOTHING_WINDOW = 5
DEFAULT_PEAK_PROMINENCE = 0.05
DEFAULT_PEAK_DISTANCE = 10


#### Configuration Block: `EXERCISE_CONFIG`

**Purpose:**

This dictionary (`EXERCISE_CONFIG`) defines how the primary movement trajectory should be calculated for each specific exercise during the repetition counting steps. This dictionary stores the **exercise-specific rules** for quantifying the repetitive motion, which are then used by the processing functions. You can add or modify entries here to support new exercises or change the analysis method for existing ones.

The keys of the dictionary (e.g., `'tricep_pushdown'`) should match the corresponding baseline subfolder names (pay attention to lowercase/underscores).

**Outcomes:**

Running this code block defines the `EXERCISE_CONFIG` variable in the notebook's memory.

* `EXERCISE_CONFIG`: ( `dict`)
    * **Keys:** Exercise label strings (e.g., `'squat'`).
    * **Values:** A nested dictionary containing configuration for that specific exercise. Currently, it holds the `'trajectory_logic'`.

**Types/Units/Meaning within `trajectory_logic`:**

For each exercise, the nested `'trajectory_logic'` dictionary specifies:

* `'metric'`: ( `str`)
    * **Meaning:** Defines *what measurement* represents the cyclical motion of the repetition (e.g., `'elbow_angle'`, `'hip_y'`, `'shoulder_y'`). The `_calculate_trajectory_from_config` function uses this string to select the correct calculation method.
    * Unit = Depends on the metric (e.g., degrees for angles, normalized Y-coordinate units for `_y` metrics).
* `'joints'`: ( `list` of `str`)
    * **Meaning:** Specifies the *base names* of the joints required to calculate the chosen `metric`. For metrics involving paired joints (like angles or averaging Y-coordinates), the code will automatically look for 'Left' and 'Right' versions (e.g., if 'Shoulder' is listed for a `_y` metric, it will try to find 'Left Shoulder' and 'Right Shoulder'). For angle metrics needing 3 joints, list them in order (e.g., `['Shoulder', 'Elbow', 'Wrist']`).
    * Unit = Joint names (strings).
* `'invert_for_valley'`: ( `bool`)
    * **Meaning:** This crucial flag tells the peak-finding algorithm whether the defining moment of a repetition cycle corresponds to a *minimum* value (`True`) or a *maximum* value (`False`) in the calculated `trajectory` signal.
        * Set to `True` if the lowest point marks the rep boundary (e.g., bottom of a squat, lowest point of a dip, most extended arm in a bicep curl). The code will invert the trajectory so `find_peaks` looks for these valleys.
        * Set to `False` if the highest point marks the rep boundary (e.g., fully extended arm in a tricep pushdown). The code will use the trajectory directly with `find_peaks`.
    * Unit = Boolean.

In [223]:
EXERCISE_CONFIG = {
    'tricep_pushdown': {
        'trajectory_logic': {
            'metric': 'elbow_angle', # Use 'elbow_angle' # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Elbow', 'Wrist'], # Base joint names
            'invert_for_valley': False # Peak angle marks end of repetition
        },
    },
    'tricep_dips': {
        'trajectory_logic': {
            'metric': 'shoulder_y', # Use Y-coordinate of shoulders # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder'], # Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
        },
    },
    'bicep_curl': {
         'trajectory_logic': {
            'metric': 'elbow_angle', # Use 'elbow_angle' # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Elbow', 'Wrist'],# Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
         },
    },
    'squat': {
         'trajectory_logic': {
             'metric': 'hip_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
             'joints': ['Hip'], # Base joint names
             'invert_for_valley': True # Lowest point (valley) marks end of repetition
         }
    },
    # 'lunges': {
    #    'trajectory_logic': {
    #        'metric': 'knee_angle',
    #        'joints': ['Hip', 'Knee', 'Ankle'],
    #        'invert_for_valley': True
    #     },
    # },
}

EXERCISE_CONFIG loaded


### Model Setup

**Purpose:**

This cell prepares and loads the core pose estimation model (`SimpleHRNet`). It first determines whether to use the GPU (`cuda`) or CPU for computation based on availability. Then, it attempts to load the pre-trained SimpleHRNet model weights from the file specified by `MODEL_PATH` (defined in Cell 2). It intelligently infers the model size parameter (`c=48` or `c=32`) from the weights filename. Crucially, it includes error handling to catch common issues like the weights file being missing or problems during model initialization, and it will stop execution if the model cannot be loaded successfully.

**Outcomes:**

Running this cell defines and potentially initializes the following:

* `DEVICE`: ( `torch.device`)
    * **Meaning:** Represents the hardware device (CPU or CUDA-enabled GPU) that the model will run on. Determined automatically based on `torch.cuda.is_available()`.
    * Unit = Device identifier.
* `HRNET_MODEL`: ( `SimpleHRNet` object or `None`)
    * **Meaning:** If loading is successful, this variable holds the initialized SimpleHRNet model object, ready for pose prediction. If loading fails for any reason, it remains `None`.
    * Unit = N/A (Python object).

**Printed Output:**

* Indicates which device (`cpu` or `cuda`) is being used.
* Confirms successful model loading or prints specific error messages if loading fails (e.g., "File not found", "SimpleHRNet class not found", or other exceptions).
* If loading fails, prints "Halting execution..." and raises a `RuntimeError`.

**Calculated Means / Logic:**

* **Device Selection:** Uses `torch.cuda.is_available()` to check for a usable NVIDIA GPU with CUDA drivers installed. Prefers GPU ('cuda') for faster processing if available, otherwise falls back to 'cpu'.
* **Model Size Inference:** Extracts 'w48' or assumes 'w32' from the `MODEL_PATH` filename to set the `c` parameter (model channel complexity) correctly when initializing `SimpleHRNet`.
* **File Existence Check:** Explicitly checks `MODEL_PATH.exists()` before attempting to load, providing a clearer error message if the weights file is missing.
* **Model Initialization:** Calls the `SimpleHRNet` constructor with necessary parameters derived from Cell 2 constants (`EXPECTED_JOINT_COUNT`, `MODEL_PATH`) and the determined `DEVICE` and `c` value. Parameters like `multiperson=False` and `max_batch_size=16` configure the prediction behavior.
* **Error Handling:** The `try...except` block catches potential errors during loading (missing files, import errors, general exceptions) and provides informative output instead of crashing abruptly. The final `if HRNET_MODEL is None:` check ensures that subsequent cells won't run if the model, which is essential, failed to load.

In [224]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

HRNET_MODEL = None
try:
    model_c_value = 48
    if not MODEL_PATH.exists():
         raise FileNotFoundError(f"Model checkpoint file not found at {MODEL_PATH}")

    HRNET_MODEL = SimpleHRNet(c=model_c_value,
                              nof_joints=EXPECTED_JOINT_COUNT,
                              checkpoint_path=MODEL_PATH,
                              device=DEVICE,
                              multiperson=False,
                              max_batch_size=16)
    print("SimpleHRNet model loaded successfully.")
except FileNotFoundError as nofile_error:
    print(nofile_error)

Using device: cpu
device: 'cpu'
SimpleHRNet model loaded successfully.


### Subfunctions

**Purpose:**

This cell defines several fundamental utility functions that perform common, low-level tasks required by the main processing functions later in the notebook. These helpers handle safe data access, geometric calculations, data cleaning, and sequence standardization.

**Functions Defined:**

1.  **`get_keypoint(frame_kps, kp_index)`:**
    * **Purpose:** To safely retrieve the valid X, Y coordinates for a *single* specified joint (`kp_index`) from the data of a single frame (`frame_kps`, assumed to be reshaped to `(num_joints, 2)`).
    * **Outcome:** The coordinates of the requested joint if found and not NaN, otherwise `None`.
    *  `numpy.array` (shape (2,)) or `None`.
    * Unit = Coordinate values (typically unitless after spatial normalization).
    * **Meaning:** Provides a robust way to access joint data for calculations like angles, preventing errors if a joint wasn't detected or is out of bounds.

2.  **`calculate_angle(p1, p2, p3)`:**
    * **Purpose:** To compute the angle (in degrees) formed at point `p2` by the vectors connecting `p1-p2` and `p3-p2`. Commonly used for joint angles (e.g., elbow angle using shoulder, elbow, wrist points).
    * **Outcome:** The calculated angle if all points are valid and form a non-zero angle, otherwise `None`.
    *  `float` or `None`.
    * Unit = Degrees.
    * **Meaning:** Quantifies the flexion or extension between three connected body parts, which is often a key indicator for specific exercise movements. Includes checks for missing points and zero-length vectors.

3.  **`handle_nan_values(sequence)`:**
    * **Purpose:** To clean a pose sequence by removing entire frames (rows) that contain *any* `NaN` (Not a Number) values. This often occurs if one or more joints were not detected in that frame.
    * **Outcome:** A `numpy.array` containing only the frames where all joints were detected. This output array might have fewer rows (frames) than the input `sequence`.
    *  `numpy.array`.
    * Unit = Same coordinate units as the input sequence.
    * **Meaning:** This function provides a simple way to ensure subsequent calculations aren't affected by missing data points. However, it can lead to data loss if joints are frequently missing for short periods.

4.  **`time_normalize_sequence(sequence, target_length)`:**
    * **Purpose:** To standardize the temporal length of a pose sequence segment (like a single repetition) to a fixed number of steps (`target_length`). It resamples the sequence using linear interpolation.
    * **Outcome:** A `numpy.array` with shape (`target_length`, number_of_coordinates), where the movement from the original `sequence` has been stretched or compressed in time.
    *  `numpy.array`.
    * Unit = Same coordinate units as the input sequence; the "time" axis is now normalized to `target_length` steps.
    * **Meaning:** Essential for comparing sequences of different original durations using DTW or for averaging multiple repetition segments together to create a baseline. It ensures comparisons focus on movement shape over a standard duration. Includes robust NaN filling using pandas `ffill`/`bfill` as a fallback after interpolation.

In [225]:
def get_keypoint(frame_kps, kp_index):
    """Safely get keypoint coordinates (x, y) from a frame's keypoint array."""
    if frame_kps is not None and 0 <= kp_index < frame_kps.shape[0]:
        coords = frame_kps[kp_index]
        if not np.isnan(coords).any():
            return coords
    return None

def calculate_angle(p1, p2, p3):
    """Calculates the angle (in degrees) at p2 formed by p1-p2-p3."""
    if p1 is None or p2 is None or p3 is None: return None
    v1, v2 = np.array(p1) - np.array(p2), np.array(p3) - np.array(p2)
    mag1, mag2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if mag1 * mag2 == 0 or np.isclose(mag1 * mag2, 0): return None
    cos_angle = np.clip(np.dot(v1, v2) / (mag1 * mag2), -1.0, 1.0)
    angle = np.degrees(np.arccos(cos_angle))
    return angle

def handle_nan_values(sequence):
    """Handles NaN values in a pose sequence (frames, coords). Removes frames containing any NaNs."""
    if sequence is None or sequence.ndim != 2 or sequence.shape[0] == 0:
        return np.empty((0, sequence.shape[1] if sequence is not None and sequence.ndim == 2 else 0))
    valid_frames_mask = ~np.isnan(sequence).any(axis=1)
    return sequence[valid_frames_mask]

def time_normalize_sequence(sequence, target_length):
    """Resamples a pose sequence (frames, coords) to a target length using linear interpolation."""
    if sequence is None or sequence.shape[0] < 2:
        num_coords = sequence.shape[1] if sequence is not None and sequence.ndim == 2 else EXPECTED_JOINT_COUNT * 2
        return np.full((target_length, num_coords), np.nan)

    num_frames, num_coords = sequence.shape
    original_indices = np.linspace(0, num_frames - 1, num_frames)
    target_indices = np.linspace(0, num_frames - 1, target_length)
    normalized_sequence = np.zeros((target_length, num_coords))

    for j in range(num_coords):
        valid_mask = ~np.isnan(sequence[:, j])
        if np.sum(valid_mask) < 2:
            normalized_sequence[:, j] = np.nan
        else:
            try:
                normalized_sequence[:, j] = np.interp(target_indices, original_indices[valid_mask], sequence[valid_mask, j])
            except Exception: normalized_sequence[:, j] = np.nan

    if np.isnan(normalized_sequence).any():
        try:
             df = pd.DataFrame(normalized_sequence)
             df.ffill(inplace=True)
             df.bfill(inplace=True)
             normalized_sequence = df.to_numpy()
        except NameError:
             normalized_sequence = np.nan_to_num(normalized_sequence, nan=0.0)

    return np.nan_to_num(normalized_sequence, nan=0.0)

### Angle Detection & Imports for Core Functions

**Purpose:**

This cell defines the `detect_camera_angle` function, which attempts to automatically estimate the camera's viewing angle relative to the person in the video ('left', 'right', or 'front'). It uses a heuristic (rule-of-thumb) approach based on the average horizontal positions of the shoulders. This cell also includes necessary imports for the functions defined across all subsequent Cell 5 sub-cells.

**Functions Defined:**

1.  **`detect_camera_angle(pose_sequence, joint_names, min_valid_frames=10, width_threshold_factor=0.35)`:**
    * **Purpose:** To estimate the camera angle ('left', 'right', 'front') by analyzing the average horizontal (X-coordinate) positions of the 'Left Shoulder' and 'Right Shoulder' joints over time. It assumes that in a side view, one shoulder will appear consistently further left or right on screen than the other. **Note:** This is a heuristic and may require tuning or a more sophisticated approach for high accuracy.
    * **Arguments:**
        * `pose_sequence`: ( `numpy.array`, Unit = Normalized coordinates) - The input pose data, expected shape (frames, num_coords). Should ideally be spatially normalized.
        * `joint_names`: ( `list` of `str`) - Ordered list of joint names.
        * `min_valid_frames`: ( `int`, Unit = Frames) - The minimum number of frames where *both* shoulders must be detected successfully (not NaN) for the angle detection to proceed.
        * `width_threshold_factor`: ( `float`, Unit = Unitless ratio) - A factor determining sensitivity. The absolute difference in average shoulder X-positions must exceed this fraction of the average detected shoulder width to be classified as a side view.
    * **Outcome:** A string indicating the estimated angle.
    *  `str`.
    * Unit = Category label ('left', 'right', 'front', 'unknown').
    * **Calculated Means / Logic:**
        * Identifies frames where both left and right shoulders are reliably detected (not NaN).
        * Returns 'unknown' if there are too few such reliable frames (`< min_valid_frames`).
        * Calculates the average X-position of each shoulder across the reliable frames (`avg_x_left`, `avg_x_right`).
        * Calculates the average horizontal distance between the shoulders (`avg_shoulder_width`) in reliable frames, using this as a dynamic reference scale.
        * Calculates the difference (`x_difference = avg_x_right - avg_x_left`).
        * Compares the absolute `x_difference` to a threshold (`avg_shoulder_width * width_threshold_factor`).
        * If the difference is large and positive (right shoulder significantly further right on screen), returns 'right' (camera view from the right).
        * If the difference is large and negative (right shoulder significantly further left on screen), returns 'left' (camera view from the left).
        * Otherwise (shoulders relatively aligned horizontally), returns 'front'.
        * Returns 'unknown' if necessary joints aren't found or other errors occur.

In [226]:
def detect_camera_angle(pose_sequence, joint_names, min_valid_frames=10, width_threshold_factor=0.35):
    """
    Analyzes pose sequence for camera angle based on shoulder positions.
    Placeholder heuristic - needs validation and likely improvement.
    """
    if pose_sequence is None or pose_sequence.ndim != 2 or pose_sequence.shape[0] < min_valid_frames:
        return 'unknown'

    try:
        lshoulder_idx = joint_names.index('Left Shoulder')
        rshoulder_idx = joint_names.index('Right Shoulder')
        lshoulder_x_idx, rshoulder_x_idx = 2 * lshoulder_idx, 2 * rshoulder_idx

        if max(lshoulder_x_idx, rshoulder_x_idx) >= pose_sequence.shape[1]:
             return 'unknown'

        valid_mask = ~np.isnan(pose_sequence[:, lshoulder_x_idx]) & \
                     ~np.isnan(pose_sequence[:, rshoulder_x_idx])

        if np.sum(valid_mask) < min_valid_frames:
            return 'unknown'

        valid_lsx = pose_sequence[valid_mask, lshoulder_x_idx]
        valid_rsx = pose_sequence[valid_mask, rshoulder_x_idx]

        avg_x_left = np.mean(valid_lsx)
        avg_x_right = np.mean(valid_rsx)

        avg_shoulder_width = np.mean(np.abs(valid_lsx - valid_rsx))

        if avg_shoulder_width < 1e-6:
             return 'front'

        x_difference = avg_x_right - avg_x_left

        threshold = avg_shoulder_width * width_threshold_factor

        detected_angle = 'front'
        if x_difference > threshold:
            detected_angle = 'right'
        elif x_difference < -threshold:
            detected_angle = 'left'

        return detected_angle

    except (ValueError, IndexError):
        return 'unknown'
    except Exception as e:
        return 'unknown'

### Video Processing en Normalization

**Purpose:**

This cell defines two crucial functions for the initial processing stages: `process_single_video` extracts the raw pose data from a video file using the loaded HRNet model, and `normalize_pose_sequence` performs spatial normalization on the extracted raw poses to make them comparable.

**Functions Defined:**

1.  **`process_single_video(video_path, model, expected_joint_count)`:**
    * **Purpose:** To read a specified video file frame-by-frame, apply the loaded pose estimation `model` (`HRNET_MODEL`) to each frame, and extract the raw joint coordinates. It assumes a single person is the primary subject.
    * **Outcome:** Returns a 2D NumPy array where each row represents a frame and columns contain the flattened (x, y) coordinates for all `expected_joint_count` joints (e.g., shape `(num_frames, 34)` for 17 joints). If pose detection fails on a frame, or the wrong number of joints/coords are detected, that frame's row will be filled with `NaN` values. Returns `None` if the video file cannot be opened or no frames are processed.
    *  `numpy.array` or `None`.
    * Unit = Pixel coordinates (relative to the video frame dimensions).
    * **Meaning:** This function translates the visual information in the video into a time series of raw skeletal joint positions. It's the first step in converting the video into analyzable data.

2.  **`normalize_pose_sequence(pose_sequence, joint_names, ref_joint1_name="Left Shoulder", ref_joint2_name="Right Shoulder")`:**
    * **Purpose:** To perform spatial normalization on the raw pose sequence obtained from `process_single_video`. This aims to make the pose data invariant to the person's location within the video frame and their scale (distance from the camera).
    * **Mechanism:** For each frame, it calculates a center point (midpoint between `ref_joint1` and `ref_joint2`, typically Left/Right Shoulder) and a scale factor (distance between `ref_joint1` and `ref_joint2`). It then recalculates all joint coordinates relative to this center point and scales them by the calculated scale factor.
    * **Outcome:** Returns a `numpy.array` of the same shape as the input `pose_sequence`, but containing the *normalized* coordinates. If the reference joints cannot be found in a frame or the scale factor is zero, that frame's row will be filled with `NaN` values.
    *  `numpy.array`.
    * Unit = Unitless normalized coordinates. The values represent positions relative to the reference joints, scaled by the distance between them.
    * **Meaning:** This step is crucial for reliable comparison (like DTW). It ensures that the subsequent analysis focuses on the *shape* and *dynamics* of the movement, rather than being affected by whether the person was standing further left/right or closer/further from the camera in different videos.

In [227]:
def process_single_video(video_path, model, expected_joint_count=EXPECTED_JOINT_COUNT):
    """Processes a single video, extracts joints, returns NumPy array (frames, num_coords)."""
    if not os.path.exists(video_path): print(f"E: Video not found: {video_path}"); return None
    video = cv2.VideoCapture(video_path);
    if not video.isOpened(): print(f"E: Cannot open video: {video_path}"); return None
    video_joints, frame_count = [], 0; expected_coords = expected_joint_count * 2
    try:
        while True:
            ret, frame = video.read();
            if not ret: break
            try:
                joints = model.predict(frame)
                if joints is None or joints.shape[0]==0 or joints.shape[1]!=expected_joint_count: frame_coords=np.full(expected_coords,np.nan)
                else: frame_coords=joints[0,:,:2].flatten(); frame_coords = frame_coords if frame_coords.shape[0]==expected_coords else np.full(expected_coords,np.nan)
            except Exception as e: frame_coords = np.full(expected_coords, np.nan)
            video_joints.append(frame_coords); frame_count += 1
    finally: video.release()
    return np.array(video_joints) if video_joints else None

def normalize_pose_sequence(pose_sequence, joint_names, ref_joint1_name="Left Shoulder", ref_joint2_name="Right Shoulder"):
    """Spatially normalizes a pose sequence based on reference joints."""
    if pose_sequence is None or pose_sequence.shape[0] == 0: return np.array([])
    try: ref_joint1_idx, ref_joint2_idx = joint_names.index(ref_joint1_name), joint_names.index(ref_joint2_name)
    except ValueError: print(f"E: Ref joints not found in normalize"); return np.full_like(pose_sequence, np.nan)
    normalized_frames = []; num_coords = pose_sequence.shape[1]
    if num_coords == 0 or num_coords % 2 != 0 or num_coords != len(joint_names) * 2:
        print(f"W: Coord mismatch/zero coords in normalize_pose_sequence ({num_coords} vs {len(joint_names)*2}). Returning NaNs.")
        return np.full_like(pose_sequence, np.nan)

    for frame in pose_sequence:
        try:
             if frame.shape[0] != num_coords: normalized_frames.append(np.full(num_coords, np.nan)); continue
             frame_joints = frame.reshape(-1, 2);
             if frame_joints.shape[0] != len(joint_names): normalized_frames.append(np.full(num_coords, np.nan)); continue
             ref1, ref2 = get_keypoint(frame_joints, ref_joint1_idx), get_keypoint(frame_joints, ref_joint2_idx)
             if ref1 is None or ref2 is None: normalized_frames.append(np.full(num_coords, np.nan)); continue
             center, scale = (np.array(ref1) + np.array(ref2)) / 2.0, np.linalg.norm(np.array(ref1) - np.array(ref2))
             if scale == 0 or np.isclose(scale, 0) or np.isnan(scale): normalized_frames.append(np.full(num_coords, np.nan)); continue
             normalized_frames.append(((frame_joints - center) / scale).flatten())
        except Exception as e: normalized_frames.append(np.full(num_coords, np.nan)); continue
    return np.array(normalized_frames)

Video processing and normalization functions


### Trajectory Calculation

**Purpose:**

This cell is to take a pose sequence and, based on the specific configuration defined for an exercise in `EXERCISE_CONFIG`, calculate a 1-dimensional time series (the "trajectory") that represents the key repetitive movement of that exercise. This trajectory is the signal that will later be analyzed (smoothed and peak-detected) for segmentation and repetition counting.

**Functions Defined:**

1.  **`_calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_config)`:**
    * **Purpose:** To calculate the 1D trajectory signal according to the rules specified in the `trajectory_config` dictionary (which comes from the main `EXERCISE_CONFIG`). It handles different calculation methods ('metrics') like joint angles or coordinate positions and attempts to use both left and right sides of the body where appropriate.
    * **Arguments:**
        * `pose_sequence`: ( `numpy.array`, Unit = Normalized coordinates) - The cleaned, spatially normalized pose sequence (frames, coords).
        * `joint_names`: ( `list` of `str`) - Ordered list of joint names.
        * `trajectory_config`: ( `dict`) - The specific `trajectory_logic` dictionary for the current exercise, extracted from `EXERCISE_CONFIG`. Contains keys like `'metric'`, `'joints'`, `'invert_for_valley'`.
    * **Outcome:** Returns a tuple containing:
        * `trajectory`: ( `numpy.array` or `None`) - A 1D NumPy array of shape `(num_frames,)` representing the calculated metric over time. Contains `NaN` values filled via interpolation and mean/zero-filling. Returns `None` if calculation fails.
        * `invert_for_valley`: ( `bool`) - The boolean flag directly passed through from the input `trajectory_config`, indicating if the peak finding logic should invert this trajectory to find valleys.
    * Unit = Depends on the `'metric'` defined in the config (e.g., degrees for `_angle` metrics, normalized units for `_y` or `_x` metrics).
    * **Calculated Means / Logic:**
        * **Configuration Parsing:** Reads the `'metric'`, `'joints'`, and `'invert_for_valley'` keys from the input `trajectory_config`.
        * **Metric-Based Calculation:**
            * **`_angle` metrics:** Expects 3 base joint names. It attempts to find corresponding Left and Right joints (e.g., 'Left Shoulder', 'Right Shoulder'). It calculates the angle for each side frame-by-frame using `calculate_angle` (handling missing points). The final trajectory is the average of the left and right angles for each frame (using `np.nanmean`, which intelligently handles frames where only one side is valid). Includes a fallback to calculate using non-paired joints if L/R versions aren't found.
            * **`_y` / `_x` metrics:** Expects 1 base joint name. It attempts to find the Left and Right corresponding joints and extracts the specified coordinate (Y or X). The final trajectory is the average of the left and right coordinates for each frame (using `np.nanmean`). Includes a fallback to use a single non-paired joint if L/R versions aren't found.
        * **Post-Processing:** After calculating the raw trajectory based on the metric, it performs robust NaN handling: first interpolating missing values, then filling any remaining NaNs (e.g., at the ends) with the mean or zero. This ensures the peak finding function receives a clean, complete signal.
        * **Error Handling:** Includes checks for missing config keys, incorrect number of joints for a metric, and errors during joint index lookups or calculations, returning `None, False` upon failure.

In [228]:
def _calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_config):
    """Calculates the 1D trajectory signal based on configuration."""
    if pose_sequence is None or pose_sequence.ndim!= 2 or pose_sequence.shape[0] < 2: return None, False

    metric = trajectory_config.get('metric', 'unknown')
    base_joint_names = trajectory_config.get('joints', [])
    invert_for_valley = trajectory_config.get('invert_for_valley', False)
    trajectory = None

    try:
        if not base_joint_names: raise ValueError("No 'joints' specified in trajectory_config")

        if metric.endswith('_angle'):
            if len(base_joint_names) != 3: raise ValueError(f"{metric} needs 3 joints in config")
            j1_base, j2_base, j3_base = base_joint_names[0], base_joint_names[1], base_joint_names[2]
            angles_left, angles_right = [], []
            try: 
                l_j1_idx = joint_names.index(f'Left {j1_base}')
                l_j2_idx = joint_names.index(f'Left {j2_base}')
                l_j3_idx = joint_names.index(f'Left {j3_base}')
                r_j1_idx = joint_names.index(f'Right {j1_base}')
                r_j2_idx = joint_names.index(f'Right {j2_base}')
                r_j3_idx = joint_names.index(f'Right {j3_base}')
                indices_valid = True
            except (ValueError, IndexError): indices_valid = False # L/R pair doesn't exist

            if indices_valid:
                for frame_coords in pose_sequence:
                    f_j = frame_coords.reshape(-1, 2)
                    p1l,p2l,p3l = get_keypoint(f_j,l_j1_idx), get_keypoint(f_j,l_j2_idx), get_keypoint(f_j,l_j3_idx)
                    p1r,p2r,p3r = get_keypoint(f_j,r_j1_idx), get_keypoint(f_j,r_j2_idx), get_keypoint(f_j,r_j3_idx)
                    angles_left.append(calculate_angle(p1l, p2l, p3l) if all(p is not None for p in [p1l, p2l, p3l]) else np.nan)
                    angles_right.append(calculate_angle(p1r, p2r, p3r) if all(p is not None for p in [p1r, p2r, p3r]) else np.nan)
                traj_l, traj_r = np.array(angles_left), np.array(angles_right)
                trajectory = np.nanmean([traj_l, traj_r], axis=0) 
            else: 
                 j1_idx, j2_idx, j3_idx = joint_names.index(j1_base), joint_names.index(j2_base), joint_names.index(j3_base)
                 angles = []
                 for frame_coords in pose_sequence:
                      f_j = frame_coords.reshape(-1, 2)
                      p1, p2, p3 = get_keypoint(f_j, j1_idx), get_keypoint(f_j, j2_idx), get_keypoint(f_j, j3_idx)
                      angles.append(calculate_angle(p1, p2, p3) if all(p is not None for p in [p1, p2, p3]) else np.nan)
                 trajectory = np.array(angles)

        elif metric.endswith('_y') or metric.endswith('_x'):
            coord_idx = 1 if metric.endswith('_y') else 0 
            base_joint = base_joint_names[0]
            try: 
                 l_joint_idx = joint_names.index(f'Left {base_joint}')
                 r_joint_idx = joint_names.index(f'Right {base_joint}')
                 if max(2*l_joint_idx+coord_idx, 2*r_joint_idx+coord_idx) >= pose_sequence.shape[1]: raise ValueError("Joint index out of bounds")
                 l_coord = pose_sequence[:, 2 * l_joint_idx + coord_idx]
                 r_coord = pose_sequence[:, 2 * r_joint_idx + coord_idx]
                 trajectory = np.nanmean([l_coord, r_coord], axis=0)
            except (ValueError, IndexError):
                 joint_idx = joint_names.index(base_joint)
                 if 2*joint_idx+coord_idx >= pose_sequence.shape[1]: raise ValueError("Joint index out of bounds")
                 trajectory = pose_sequence[:, 2 * joint_idx + coord_idx]
        else:
             raise ValueError(f"Unknown metric type specified in config: '{metric}'")

        if trajectory is None or trajectory.shape[0] == 0 or np.all(np.isnan(trajectory)): return None, False
        nan_mask = np.isnan(trajectory)
        if np.any(nan_mask):
            indices = np.arange(len(trajectory)); valid_indices = np.flatnonzero(~nan_mask)
            if len(valid_indices) < 2: return None, False
            trajectory[nan_mask] = np.interp(indices[nan_mask], valid_indices, trajectory[valid_indices])
        if np.any(np.isnan(trajectory)):
             trajectory = np.nan_to_num(trajectory, nan=np.nanmean(trajectory))
             if np.any(np.isnan(trajectory)): trajectory = np.nan_to_num(trajectory, nan=0.0)


        return trajectory, invert_for_valley

    except (ValueError, IndexError) as e: print(f"W: Error finding/using joints for metric '{metric}': {e}"); return None, False
    except Exception as e: print(f"E: Error calculating trajectory with metric '{metric}': {e}"); return None, False

Trajectory calculation function loaded


### Peak Finding Helper Function

**Purpose:**

This cell is to take a 1D numerical trajectory (representing movement over time, like an angle or position) and identify the locations (frame indices) of significant peaks within that signal. By using the `invert_trajectory` flag, it can also be used to find significant valleys (minima). This peak/valley detection is used for identifying individual repetition boundaries in the functions `segment_repetitions` and `count_repetitions`.

**Functions Defined:**

1.  **`_extract_trajectory_and_find_peaks(trajectory, smoothing_window, prominence, distance, invert_trajectory=False)`:**
    * **Purpose:** To smooth an input trajectory signal and find the indices of peaks that meet specific prominence and distance criteria.
    * **Arguments:**
        * `trajectory`: ( `numpy.array`, Unit = Varies - e.g., degrees, normalized units) - The 1D input signal calculated by `_calculate_trajectory_from_config`.
        * `smoothing_window`: ( `int`, Unit = Frames) - The number of frames to use for the moving average filter applied before peak detection.
        * `prominence`: ( `float`, Unit = Unitless ratio) - Relative prominence factor. A peak must stand out from its surrounding troughs by at least this fraction of the smoothed signal's overall range.
        * `distance`: ( `int`, Unit = Frames) - The minimum number of frames required between consecutive detected peaks.
        * `invert_trajectory`: ( `bool`) - If `True`, the trajectory is inverted (`-trajectory`) before smoothing and peak finding. This effectively finds significant valleys (local minima) in the original signal.
    * **Outcome:** A 1D NumPy array containing the integer frame indices where valid peaks were detected in the *original* (un-smoothed) trajectory's timeline, or `None` if the input is invalid, smoothing/peak finding fails, or no peaks are found.
    *  `numpy.array` (int) or `None`.
    * Unit = Frame index (unitless count).
    * **Calculated Means / Logic:**
        * **Input Validation:** Checks if the input `trajectory` is valid and long enough for the specified smoothing and distance parameters.
        * **Inversion:** Optionally inverts the signal using `processed_trajectory = -trajectory` if `invert_trajectory` is `True`, allowing the same peak-finding logic to find valleys.
        * **Smoothing:** Applies a moving average filter (`np.convolve` with `mode='valid'`) of size `smoothing_window` to reduce noise and make peak detection more stable.
        * **Dynamic Prominence:** Calculates the required absolute `prominence` for `find_peaks` based on the input `prominence` factor (relative) and the peak-to-peak `data_range` of the *smoothed* signal. Ensures a minimum small prominence even for flat signals.
        * **Peak Detection:** Uses `scipy.signal.find_peaks` on the smoothed signal, filtering peaks based on the calculated `required_prominence` and the minimum horizontal `distance` between them.
        * **Index Correction:** Adjusts the indices found in the *smoothed* signal back to correspond to the indices in the *original* input `trajectory` by accounting for the offset introduced by the 'valid' convolution mode.
        * **Bounds Checking:** Ensures returned indices are valid within the length of the original trajectory.
        * **Error Handling:** Returns `None` if errors occur during the process.

In [229]:
def _extract_trajectory_and_find_peaks(trajectory, smoothing_window, prominence, distance, invert_trajectory=False):
    """Internal helper to smooth a trajectory and find peaks."""
    if trajectory is None or trajectory.ndim != 1 or trajectory.shape[0] < max(distance * 2, smoothing_window, 1): return None

    processed_trajectory = trajectory.copy()
    if invert_trajectory: processed_trajectory = -processed_trajectory
    if len(processed_trajectory) < smoothing_window: return None

    smoothed_trajectory = np.convolve(processed_trajectory, np.ones(smoothing_window)/smoothing_window, mode='valid')
    if smoothed_trajectory.shape[0] < max(distance, 1): return None

    data_range = np.ptp(smoothed_trajectory)
    required_prominence = max(data_range * prominence, 1e-6) if data_range > 1e-9 else 1e-6

    try:
        smoothed_indices_offset = (len(processed_trajectory) - len(smoothed_trajectory)) // 2
        peak_indices_smoothed, _ = find_peaks(smoothed_trajectory, prominence=required_prominence, distance=distance)
        peak_indices_original = peak_indices_smoothed + smoothed_indices_offset
        peak_indices_original = peak_indices_original[(peak_indices_original >= 0) & (peak_indices_original < len(processed_trajectory))]
        return peak_indices_original
    except Exception as e: print(f"W: Peak finding failed: {e}"); return None

Peak finding function defined


### Segmentation and Counting

**Purpose:**

This cell is analyzing the movement trajectory to identify repetitions: `segment_repetitions` breaks a sequence into individual repetition clips, and `count_repetitions` estimates the total number of repetitions performed.

**Functions Defined:**

1.  **`segment_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, smoothing_window=DEFAULT_SMOOTHING_WINDOW, peak_prominence=DEFAULT_PEAK_PROMINENCE, peak_distance=DEFAULT_PEAK_DISTANCE)`:**
    * **Purpose:** To divide a given `pose_sequence` (assumed to contain multiple repetitions of the specified `exercise_label`) into a list of smaller sequences, where each smaller sequence corresponds to a single detected repetition.
    * **Arguments:**
        * `pose_sequence`: ( `numpy.array`, Unit = Normalized coordinates) - The input sequence to segment.
        * `exercise_label`: ( `str`) - The name of the exercise, used to look up the correct processing logic in `exercise_config`.
        * `joint_names`: ( `list` of `str`) - Ordered list of joint names.
        * `exercise_config`: ( `dict`) - The main configuration dictionary containing trajectory logic for all exercises.
        * `smoothing_window`, `peak_prominence`, `peak_distance`: ( `int`/`float`, Unit = Frames/Unitless ratio) - Parameters controlling the peak detection process (currently uses defaults defined in Cell 2, but could be overridden).
    * **Outcome:** A list, where each element is a `numpy.array` representing the pose data for a single detected repetition segment (shape `(segment_frames, num_coords)`). Returns an empty list if no valid repetitions (at least 2 peaks) are found or if the config is missing.
    *  `list` of `numpy.array` (float).
    * Unit = Each array contains normalized coordinates.
    * **Meaning:** This function isolates the individual motion cycles (reps) within a longer performance. This is crucial for creating the averaged baselines (where each element being averaged should be one rep) and for the per-repetition analysis during recognition. It uses the default peak finding parameters to consistently extract segments for baseline creation.

2.  **`count_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, prominence=DEFAULT_PEAK_PROMINENCE, distance=DEFAULT_PEAK_DISTANCE, smoothing_window=DEFAULT_SMOOTHING_WINDOW)`:**
    * **Purpose:** To estimate the total number of repetitions within the input `pose_sequence` for the given `exercise_label`. It allows passing specific `prominence` and `distance` parameters, typically the *tuned* values found during baseline loading.
    * **Arguments:**
        * `pose_sequence`: ( `numpy.array`, Unit = Normalized coordinates) - The input sequence to analyze.
        * `exercise_label`: ( `str`) - The name of the exercise.
        * `joint_names`: ( `list` of `str`) - Ordered list of joint names.
        * `exercise_config`: ( `dict`) - The main configuration dictionary.
        * `prominence`, `distance`, `smoothing_window`: ( `float`/`int`, Unit = Unitless ratio/Frames) - Peak detection parameters. Crucially, *tuned* values for `prominence` and `distance` should ideally be passed here when counting reps for a recognized exercise.
    * **Outcome:** The estimated number of repetitions detected in the sequence. Returns 0 if the config is missing or no peaks are found.
    *  `int`.
    * Unit = Count (unitless).
    * **Meaning:** Provides the final repetition count estimate. Its accuracy depends heavily on the quality of the pose data, the appropriateness of the trajectory logic defined in `EXERCISE_CONFIG`, and the effectiveness of the (potentially tuned) `prominence` and `distance` parameters used.

In [230]:
def segment_repetitions(pose_sequence, exercise_label, joint_names, exercise_config,
                        smoothing_window=DEFAULT_SMOOTHING_WINDOW,
                        peak_prominence=DEFAULT_PEAK_PROMINENCE,
                        peak_distance=DEFAULT_PEAK_DISTANCE):
    """Segments a pose sequence into individual repetitions using EXERCISE_CONFIG."""
    current_exercise_cfg_dict = exercise_config.get(exercise_label, {})
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg:
        return []

    trajectory, invert = _calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_logic_cfg)
    peak_indices = _extract_trajectory_and_find_peaks(trajectory, smoothing_window, peak_prominence, peak_distance, invert_trajectory=invert)

    if peak_indices is None or len(peak_indices) < 2: return []
    repetitions = [pose_sequence[peak_indices[i]:peak_indices[i+1], :] for i in range(len(peak_indices) - 1)]
    return [rep for rep in repetitions if rep.shape[0] > 1]

def count_repetitions(pose_sequence, exercise_label, joint_names, exercise_config,
                      prominence=DEFAULT_PEAK_PROMINENCE, distance=DEFAULT_PEAK_DISTANCE,
                      smoothing_window=DEFAULT_SMOOTHING_WINDOW):
    """Counts repetitions using EXERCISE_CONFIG and specified parameters."""
    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    if not current_exercise_cfg_dict: return 0
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg: return 0

    trajectory, invert = _calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_logic_cfg)
    peak_indices = _extract_trajectory_and_find_peaks(trajectory, smoothing_window, prominence, distance, invert_trajectory=invert)

    return len(peak_indices) if peak_indices is not None else 0

Segmentation and counting functions defined 


In [231]:
def tune_counting_parameters(exercise_label, labeled_video_data, joint_names, exercise_config,
                             prominence_range, distance_range):
    """Performs grid search using the refactored peak finding and config."""
    print(f"  Tuning repetition counting parameters for: {exercise_label}...")
    best_params = (DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE) # Start with defaults
    min_total_error = float('inf')
    if not labeled_video_data: print("W: No labeled data provided for tuning."); return best_params

    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    if not current_exercise_cfg_dict: print(f"W: No config for {exercise_label} in tuning"); return best_params
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg: print(f"W: No trajectory_logic in config for {exercise_label}"); return best_params

    num_combinations = len(prominence_range) * len(distance_range)
    default_error = 0
    precalculated_trajectories = {}

    for i, (vid_idx, (sequence, true_reps)) in enumerate(labeled_video_data): 
        if sequence is None or sequence.ndim != 2 or sequence.shape[0] < 2: continue
        trajectory, invert = _calculate_trajectory_from_config(sequence, joint_names, trajectory_logic_cfg)
        if trajectory is not None: precalculated_trajectories[vid_idx] = (trajectory, invert, true_reps)

    if not precalculated_trajectories: print("W: No valid trajectories calculated for tuning."); return best_params

    for vid_idx, (traj, inv, true_r) in precalculated_trajectories.items():
         peak_indices_def = _extract_trajectory_and_find_peaks(traj, DEFAULT_SMOOTHING_WINDOW, DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE, invert_trajectory=inv)
         calc_reps_def = len(peak_indices_def) if peak_indices_def is not None else 0
         default_error += abs(calc_reps_def - true_r)

    # Grid search
    for p_idx, p in enumerate(prominence_range):
        for d_idx, d in enumerate(distance_range):
            current_total_error = 0
            for vid_idx, (traj, inv, true_r) in precalculated_trajectories.items():
                peak_indices = _extract_trajectory_and_find_peaks(traj, DEFAULT_SMOOTHING_WINDOW, p, d, invert_trajectory=inv)
                calculated_reps = len(peak_indices) if peak_indices is not None else 0
                current_total_error += abs(calculated_reps - true_r)

            if current_total_error < min_total_error:
                min_total_error = current_total_error
                best_params = (p, d)
            if min_total_error == 0: break
        if min_total_error == 0: break

    print(f"Tuning complete. Best Params: P={best_params[0]:.3f}, D={best_params[1]}, Min Error={min_total_error}. (Default Error={default_error})")

    if precalculated_trajectories:
        print(f"Detected reps using BEST params (P={best_params[0]:.3f}, D={best_params[1]}):")
        total_calculated, total_true = 0, 0

        for vid_idx, (traj, inv, true_r) in sorted(precalculated_trajectories.items()):
            peak_indices = _extract_trajectory_and_find_peaks(traj, DEFAULT_SMOOTHING_WINDOW, best_params[0], best_params[1], invert_trajectory=inv)
            calculated_reps = len(peak_indices) if peak_indices is not None else 0
            print(f"Video Index {vid_idx}: Detected={calculated_reps}, True={true_r}")
            total_calculated += calculated_reps; total_true += true_r
        print(f"Total Reps: Detected={total_calculated}, True={total_true}, Overall Error={abs(total_calculated - total_true)}")

    return best_params

Parameter tuning function defined


### Parameter Tuning Function

**Purpose:**

This cell is to automatically find the optimal `peak_prominence` and `peak_distance` values for the repetition counting logic for a specific exercise. It does this by systematically trying out different parameter combinations on the baseline videos for that exercise (where the true rep count is known from the filename) and selecting the parameter pair that minimizes the total counting error.

**Functions Defined:**

1.  **`tune_counting_parameters(exercise_label, labeled_video_data, joint_names, exercise_config, prominence_range, distance_range)`:**
    * **Purpose:** To perform a grid search over `prominence_range` and `distance_range` to find the combination of `peak_prominence` and `peak_distance` that most accurately counts repetitions across the provided `labeled_video_data` for the given `exercise_label`, using the trajectory logic defined in `exercise_config`.
    * **Arguments:**
        * `exercise_label`: ( `str`) - The name of the exercise being tuned.
        * `labeled_video_data`: ( `list` of `tuple`) - Data for tuning. Each element is `(video_index, (cleaned_pose_sequence, true_reps))`. The `cleaned_pose_sequence` is the spatially normalized sequence *before* time normalization. `true_reps` is the ground truth count parsed from the filename.
        * `joint_names`: ( `list` of `str`) - Ordered list of joint names.
        * `exercise_config`: ( `dict`) - The main configuration dictionary.
        * `prominence_range`: ( `list` of `float`) - List of `peak_prominence` values to test.
        * `distance_range`: ( `list` of `int`) - List of `peak_distance` values to test.
    * **Outcome:** Returns a tuple containing the best `(prominence, distance)` parameters found for this exercise. Also prints detailed logs about the tuning process, including the best parameters, the minimum error achieved, the error using default parameters, and a comparison of detected vs. true reps for each video using the best parameters.
    *  `tuple` containing (`float`, `int`).
    * Unit = Prominence is a unitless ratio, Distance is in frames.
    * **Calculated Means / Logic:**
        * **Pre-calculation:** It first calculates the movement trajectory for each input video sequence using `_calculate_trajectory_from_config` based on the `exercise_config`. This avoids recalculating it repeatedly inside the loops.
        * **Error Metric:** The "error" for a given parameter set is defined as the **sum of absolute differences** between the number of peaks detected (`calculated_reps`) and the ground truth (`true_reps`) across all the `labeled_video_data` provided for that exercise.
        * **Grid Search:** It systematically iterates through every possible pair of (`prominence`, `distance`) values taken from the input ranges. For each pair, it calculates the total error.
        * **`Min Error`:** The lowest total error value encountered during the grid search. A value of 0 indicates a parameter set was found that perfectly matched the true counts on all provided labeled videos.
        * **`Best Params`:** The (`prominence`, `distance`) tuple that resulted in the `Min Error`.
        * **`Default Error`:** The total error calculated using the globally defined `DEFAULT_PEAK_PROMINENCE` and `DEFAULT_PEAK_DISTANCE` values. This provides a baseline to judge if the tuning was beneficial.
        * **Output Breakdown:** The final printed list (`Detected reps using BEST params...`) shows the performance of the chosen `Best Params` on each individual labeled baseline video, comparing the detected count to the true count provided in the filename.

### Loading baseline videos

**Purpose:**

This cell processing all baseline video files. It iterates through each exercise folder provided in `BASELINE_ROOT_DIR`.

1.  **Parses Information:** Extracts the ground truth repetition count (`XX`) and camera angle (`'left'`, `'right'`, or `'front'`) from each baseline video's filename (using the format `exercise_label_XXreps_angle_description.mp4`). Videos without a recognized angle are marked as `'unknown'`.
2.  **Processes Videos:** Extracts and spatially normalizes pose sequences for each video.
3.  **Segments Repetitions:** Breaks down each video's cleaned sequence into individual repetition segments using default parameters (leveraging the exercise-specific trajectory logic from `EXERCISE_CONFIG`).
4.  **Groups by Angle:** Collects the time-normalized repetition segments and the labeled data (sequence + true reps) into groups based on the `parsed_angle` for each exercise.
5.  **Tunes Parameters per Angle:** For each angle category ('left', 'right', 'front', 'unknown') that has sufficient labeled data, it calls `tune_counting_parameters` to find the optimal peak detection settings.
6.  **Averages Baselines per Angle:** For each angle category with data, it calculates and stores the average time-normalized repetition sequence.
7.  **Derives Thresholds per Angle:** For each angle category, it calculates and stores a suggested DTW recognition threshold based on the variation between individual reps and the angle-specific average.
8.  **Calculates Overall Baseline & Threshold:** After processing all angles, it averages *all* valid repetitions for an exercise to create an 'overall' baseline and calculates an 'overall' derived threshold.

**Functions Defined:**

1.  **`load_baseline_data(baseline_root_dir, hrnet_model, joint_names, target_rep_length, exercise_config, prominence_range, distance_range, threshold_std_multiplier=DEFAULT_THRESHOLD_STD_MULTIPLIER, min_reps_for_threshold=DEFAULT_MIN_REPS_FOR_THRESHOLD)`:**
    * **Purpose:** To process all baseline videos, organize data by exercise and parsed filename angle, tune counting parameters per angle, create averaged baselines per angle and overall, and derive recognition thresholds per angle and overall.
    * **Arguments:** All necessary inputs including paths, the HRNet model, configuration dictionaries, tuning ranges, and parameters for threshold calculation.
    * **Outcome:** Returns a tuple containing three nested dictionaries:
        1.  `averaged_baseline_data`: ( `dict`) Structure: `{exercise_label: {angle_key: numpy.array}}`. Contains the averaged time-normalized pose sequence for each exercise under keys for detected angles ('left', 'right', 'front', 'unknown') and 'overall'. The value might be `None` if averaging failed (e.g., no reps).
        2.  `tuned_counting_params`: ( `dict`) Structure: `{exercise_label: {angle_key: (prominence, distance)}}`. Contains the best `(prominence, distance)` tuple found for counting reps for each exercise/angle combination based on labeled data. Uses defaults if tuning fails or no labeled data exists for an angle.
        3.  `exercise_specific_thresholds`: ( `dict`) Structure: `{exercise_label: {angle_key: float or None}}`. Contains the derived DTW threshold calculated for each exercise/angle combination based on intra-exercise repetition variance. Value is `None` if calculation failed (e.g., too few reps).
    * Unit = See types above. Arrays contain normalized coordinates, parameters are floats/ints (unitless or frames), thresholds are floats (unitless distance).
    * **Calculated Means / Logic:** This function orchestrates the entire baseline creation pipeline. Key calculations include:
        * **Filename Parsing:** Uses regular expressions (`re.search`) to extract rep counts and angle labels.
        * **Data Aggregation:** Groups processed data (`labeled_data_by_angle`, `all_reps_by_angle`) based on the parsed angle.
        * **Per-Angle Tuning/Averaging/Thresholding:** Executes the `tune_counting_parameters` function, calculates `np.nanmean` for averaging, and calculates DTW distances + statistics (mean, std dev) to derive thresholds *independently* for each angle category ('left', 'right', 'front', 'unknown') within each exercise.
        * **Overall Averaging/Thresholding:** Performs a final average and threshold calculation using *all* valid repetitions collected across *all* angles for an exercise, storing results under the 'overall' key.

In [232]:
def load_baseline_data(baseline_root_dir, hrnet_model, joint_names, target_rep_length, exercise_config, # Pass config
                       prominence_range, distance_range, # Pass tuning ranges
                       threshold_std_multiplier=DEFAULT_THRESHOLD_STD_MULTIPLIER,
                       min_reps_for_threshold=DEFAULT_MIN_REPS_FOR_THRESHOLD):
    """
    Loads baselines, uses EXERCISE_CONFIG, parses angle from filename,
    tunes params per angle, averages per angle AND overall, calculates thresholds.
    Returns nested dicts: averaged_baselines, tuned_counting_params, exercise_specific_thresholds
    """
    averaged_baseline_data = {}
    tuned_counting_params = {}
    exercise_specific_thresholds = {}
    start_time_total = time.time()

    print(f"Loading Baselines (Config-Driven, Angle from Filename)... from: {baseline_root_dir}")
    if not os.path.isdir(baseline_root_dir):
        print(f"E: Baseline directory not found: {baseline_root_dir}")
        return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds

    for exercise_label_raw in sorted(os.listdir(baseline_root_dir)):
        exercise_folder_path = os.path.join(baseline_root_dir, exercise_label_raw)
        if not os.path.isdir(exercise_folder_path): continue
        exercise_label = exercise_label_raw.lower().replace(" ", "_")
        print(f"\nProcessing baseline exercise: {exercise_label} (from folder: {exercise_label_raw})")

        current_exercise_config_dict = exercise_config.get(exercise_label)
        if not current_exercise_config_dict:
             print(f"  W: No config found for '{exercise_label}'. Skipping."); continue

        all_reps_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []}
        labeled_data_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []} # Stores (vid_idx, (sequence, true_reps))
        averaged_baseline_data[exercise_label] = {}
        tuned_counting_params[exercise_label] = {}
        exercise_specific_thresholds[exercise_label] = {}

        video_files = sorted([f for f in os.listdir(exercise_folder_path) if f.lower().endswith(('.mp4', '.avi', '.mov'))])
        print(f"  Found {len(video_files)} video file(s).")

        for video_idx, video_file in enumerate(video_files):
            video_path = os.path.join(exercise_folder_path, video_file)
            true_reps, parsed_angle = None, 'unknown'
            rep_match = re.search(r'_(\d+)reps', video_file, re.IGNORECASE)
            if rep_match: true_reps = int(rep_match.group(1))
            angle_match = re.search(r'_(left|right|front)', video_file, re.IGNORECASE)
            if angle_match: parsed_angle = angle_match.group(1).lower()

            raw_seq = process_single_video(video_path, hrnet_model)
            if raw_seq is None or raw_seq.shape[0] == 0: continue
            spatially_norm_seq = normalize_pose_sequence(raw_seq, joint_names)
            if spatially_norm_seq is None or np.all(np.isnan(spatially_norm_seq)): continue
            cleaned_seq = handle_nan_values(spatially_norm_seq)
            if cleaned_seq.shape[0] < 2: continue

            if true_reps is not None:
                 labeled_data_by_angle[parsed_angle].append((video_idx, (cleaned_seq, true_reps)))

            repetitions = segment_repetitions(cleaned_seq, exercise_label, joint_names, exercise_config)
            if not repetitions: continue

            for rep_segment in repetitions:
                if rep_segment is not None and rep_segment.shape[0] >= 2:
                    time_norm_rep = time_normalize_sequence(rep_segment, target_rep_length)
                    expected_coords = len(joint_names) * 2
                    if not np.isnan(time_norm_rep).all() and time_norm_rep.shape == (target_rep_length, expected_coords):
                        all_reps_by_angle[parsed_angle].append(time_norm_rep)

        all_reps_across_angles = []
        for angle in ['left', 'right', 'front', 'unknown']:
            labeled_data_for_this_angle = labeled_data_by_angle[angle]
            normalized_reps_for_this_angle = all_reps_by_angle[angle]
            if not normalized_reps_for_this_angle: continue
            all_reps_across_angles.extend(normalized_reps_for_this_angle)

            if labeled_data_for_this_angle:
                 tuned_p, tuned_d = tune_counting_parameters(exercise_label, labeled_data_for_this_angle,
                                                            joint_names, exercise_config, prominence_range, distance_range)
                 tuned_counting_params[exercise_label][angle] = (tuned_p, tuned_d)
            else: tuned_counting_params[exercise_label][angle] = (DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE)

            reps_stack = np.stack(normalized_reps_for_this_angle, axis=0)
            averaged_sequence_angle = np.nanmean(reps_stack, axis=0)
            averaged_baseline_data[exercise_label][angle] = np.nan_to_num(averaged_sequence_angle, nan=0.0)

            derived_threshold_angle = None
            if len(normalized_reps_for_this_angle) >= min_reps_for_threshold:
                intra_distances_angle = []
                safe_avg_angle = np.nan_to_num(averaged_sequence_angle, nan=0.0)
                for norm_rep in normalized_reps_for_this_angle:
                     safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                     if safe_rep.shape == safe_avg_angle.shape:
                         try: dist, _ = fastdtw(safe_rep, safe_avg_angle, dist=euclidean); intra_distances_angle.append(dist)
                         except Exception: pass
                if len(intra_distances_angle) >= min_reps_for_threshold:
                     mean_dist, std_dist = np.mean(intra_distances_angle), np.std(intra_distances_angle)
                     derived_threshold_angle = mean_dist + threshold_std_multiplier * std_dist
            exercise_specific_thresholds[exercise_label][angle] = derived_threshold_angle

        if len(all_reps_across_angles) >= min_reps_for_threshold:
             overall_reps_stack = np.stack(all_reps_across_angles, axis=0)
             overall_averaged_sequence = np.nanmean(overall_reps_stack, axis=0)
             averaged_baseline_data[exercise_label]['overall'] = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             overall_intra_distances = []
             safe_overall_avg = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             for norm_rep in all_reps_across_angles:
                  safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                  if safe_rep.shape == safe_overall_avg.shape:
                       try: dist, _ = fastdtw(safe_rep, safe_overall_avg, dist=euclidean); overall_intra_distances.append(dist)
                       except Exception: pass
             if len(overall_intra_distances) >= min_reps_for_threshold:
                  mean_dist, std_dist = np.mean(overall_intra_distances), np.std(overall_intra_distances)
                  derived_threshold_overall = mean_dist + threshold_std_multiplier * std_dist
                  exercise_specific_thresholds[exercise_label]['overall'] = derived_threshold_overall
             else: exercise_specific_thresholds[exercise_label]['overall'] = None
        else:
             averaged_baseline_data[exercise_label]['overall'] = None
             exercise_specific_thresholds[exercise_label]['overall'] = None

    end_time_total = time.time()
    print(f"\nBaseline processing complete in {time.time() - start_time_total:.2f}s.")
    return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds

### Recognition Function

**Purpose:**

This cell takes a new video file and analyzes it against the previously loaded baseline data to classify the exercise performed in each detected repetition.

**Functions Defined:**

1.  **`recognize_exercise(new_video_path, averaged_baseline_data, hrnet_model, joint_names, exercise_config, target_rep_length, tuned_counting_params, exercise_specific_thresholds, hint_angle=None, fallback_threshold=DTW_DISTANCE_THRESHOLD)`:**
    * **Purpose:** To process a new video, segment it into repetitions (using parameters potentially tuned for an initial best guess exercise and the hinted angle), compare each time-normalized repetition segment against angle-specific (or overall) averaged baselines using DTW, apply exercise- and angle-specific (or overall/fallback) thresholds, and return the classification result for each repetition.
    * **Arguments:**
        * `new_video_path`: ( `str` or `pathlib.Path`) - Path to the video file to be analyzed.
        * `averaged_baseline_data`: ( `dict`) - Nested dictionary `{exercise: {angle: avg_sequence}}` returned by `load_baseline_data`.
        * `hrnet_model`: ( `SimpleHRNet` object) - The loaded pose estimation model.
        * `joint_names`: ( `list` of `str`) - Ordered list of joint names.
        * `exercise_config`: ( `dict`) - The main configuration dictionary defining exercise-specific logic.
        * `target_rep_length`: ( `int`, Unit = Frames) - Target length for time normalization.
        * `tuned_counting_params`: ( `dict`) - Nested dictionary `{exercise: {angle: (prominence, distance)}}` returned by `load_baseline_data`.
        * `exercise_specific_thresholds`: ( `dict`) - Nested dictionary `{exercise: {angle: threshold}}` returned by `load_baseline_data`.
        * `hint_angle`: ( `str` or `None`, Unit = Category label) - Optional hint ('left', 'right', 'front'). If provided, comparison uses this angle's baselines/thresholds. If `None`, uses 'overall' baselines/thresholds. Default is `None`.
        * `fallback_threshold`: ( `float`, Unit = Unitless Distance) - The global DTW threshold used if no specific derived threshold is available for the exercise/angle combination being compared.
    * **Outcome:** Returns a tuple containing:
        1.  `repetition_results`: ( `list` of `tuple`) - A list where each tuple represents a detected repetition: `(rep_index, matched_label, min_distance)`.
            * `rep_index`: ( `int`) 1-based index.
            * `matched_label`: ( `str`) Contains the name of the matched exercise if `min_distance` was below the relevant threshold, otherwise a string like "No Match / Inconclusive (...)" or "Error...".
            * `min_distance`: ( `float`) The lowest DTW distance found when comparing this repetition against the relevant baselines.
        2.  `num_detected_reps`: ( `int`) - The total number of segments detected in the video by `segment_repetitions`.
    * Unit = See types above. Distances are unitless. Counts are integers. Labels are strings.
    * **Calculated Means / Logic:**
        * **Initial Processing:** Reads the video, extracts raw poses, performs spatial normalization, and cleans NaN frames (`process_single_video`, `normalize_pose_sequence`, `handle_nan_values`).
        * **Initial Guess:** Time-normalizes the *entire cleaned sequence* and compares it against the 'overall' averaged baseline for each exercise to get a quick `initial_best_label`.
        * **Segmentation Parameter Selection:** Uses the `initial_best_label` and the `hint_angle` (or 'front' as fallback) to look up the *tuned* counting parameters (`segment_p`, `segment_d`) from `tuned_counting_params`. Defaults are used if tuning failed or the exercise/angle wasn't found.
        * **Test Video Segmentation:** Calls `segment_repetitions` on the *cleaned, non-time-normalized* sequence, passing the `exercise_config` (for the guessed exercise if available) and the selected `segment_p`, `segment_d` to break the video into `test_repetitions`.
        * **Angle Key Selection:** Determines the `comparison_angle_key` ('left', 'right', 'front', or 'overall') based on the `hint_angle`.
        * **Per-Repetition Comparison:** Loops through each `rep_segment`:
            * Time-normalizes the segment to `target_rep_length`.
            * Compares this normalized segment against the averaged baseline corresponding to the `comparison_angle_key` for *every* exercise using DTW.
            * Finds the exercise with the minimum DTW distance (`best_rep_label_for_this_rep`, `min_rep_distance`).
        * **Angle-Aware Thresholding:** Determines the correct threshold to use by checking `exercise_specific_thresholds` for the `best_rep_label_for_this_rep` and the `comparison_angle_key`. It includes fallbacks: first checks the specific angle threshold, then the 'overall' threshold for that exercise, then the global `fallback_threshold`.
        * **Rep Classification:** Compares `min_rep_distance` against the determined `threshold_to_use` to decide the final `matched_label_for_rep`.
        * **Result Aggregation:** Stores the results for each repetition in the `repetition_results` list.

In [233]:
def recognize_exercise(new_video_path, averaged_baseline_data, hrnet_model, joint_names, exercise_config,
                       target_rep_length, tuned_counting_params, exercise_specific_thresholds,
                       hint_angle=None, fallback_threshold=DTW_DISTANCE_THRESHOLD):
    """Recognizes exercise using angle hint, config-driven logic, angle-specific baselines/thresholds."""
    print(f"\n--- Recognizing Exercise (Angle Hint='{hint_angle}') for: {os.path.basename(new_video_path)} ---")
    start_time = time.time()
    if tuned_counting_params is None: tuned_counting_params = {}
    if exercise_specific_thresholds is None: exercise_specific_thresholds = {}
    default_p, default_d = DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE

    new_raw_seq = process_single_video(new_video_path, hrnet_model)
    if new_raw_seq is None or new_raw_seq.shape[0] == 0: print("E: Failed processing video."); return [], 0
    new_spatially_norm_seq = normalize_pose_sequence(new_raw_seq, joint_names)
    if new_spatially_norm_seq is None or np.all(np.isnan(new_spatially_norm_seq)): print("E: Failed spatial normalization."); return [], 0
    new_sequence_clean = handle_nan_values(new_spatially_norm_seq)
    if new_sequence_clean.shape[0] < 2: print("W: Sequence too short after cleaning."); return [], 0

    new_sequence_time_norm = time_normalize_sequence(new_sequence_clean, target_rep_length)
    if np.isnan(new_sequence_time_norm).all(): print("E: Time normalization failed for whole sequence."); return [], 0
    new_sequence_time_norm = np.nan_to_num(new_sequence_time_norm, nan=0.0)

    initial_min_distance, initial_best_label = float('inf'), None
    segment_p, segment_d = default_p, default_d
    segment_exercise_config_dict = None

    if not averaged_baseline_data: print("E: No baseline data."); return [], 0
    for ex_label, angle_dict in averaged_baseline_data.items():
        avg_base_seq = angle_dict.get('overall')
        if avg_base_seq is None: continue
        if avg_base_seq.shape[0] != target_rep_length or avg_base_seq.ndim != 2 or \
           new_sequence_time_norm.shape[1] != avg_base_seq.shape[1]: continue
        safe_base = np.nan_to_num(avg_base_seq, nan=0.0)
        try: dist, _ = fastdtw(new_sequence_time_norm, safe_base, dist=euclidean)
        except Exception: continue
        if dist < initial_min_distance: initial_min_distance, initial_best_label = dist, ex_label

    angle_for_tuning_lookup = hint_angle if hint_angle in ['left', 'right', 'front'] else 'front'

    if initial_best_label:
        segment_exercise_config_dict = exercise_config.get(initial_best_label)
        if segment_exercise_config_dict:
             tuned_p, tuned_d = tuned_counting_params.get(initial_best_label, {}).get(angle_for_tuning_lookup, (default_p, default_d))
             segment_p, segment_d = tuned_p, tuned_d
        else:
             print(f"W: No config found for initial guess '{initial_best_label}'. Using default segmentation params.")
             initial_best_label = None
    else:
        print(f"  Initial guess failed. Using default")

    segment_label_for_logic = initial_best_label if initial_best_label else "unknown"
    test_repetitions = segment_repetitions(new_sequence_clean, segment_label_for_logic, joint_names, exercise_config,
                                           peak_prominence=segment_p, peak_distance=segment_d)
    num_detected_reps = len(test_repetitions)
    print(f"  Detected {num_detected_reps} potential repetitions in test video.")
    if num_detected_reps == 0: return [], 0

    repetition_results = []
    comparison_angle_key = hint_angle if hint_angle in ['left', 'right', 'front'] else 'overall'
    print(f"  Comparing each detected repetition against '{comparison_angle_key}' baselines...")

    for i, rep_segment in enumerate(test_repetitions):
        if rep_segment is None or rep_segment.shape[0] < 2: repetition_results.append( (i+1, "Error: Invalid Segment", float('inf')) ); continue
        time_norm_rep_segment = time_normalize_sequence(rep_segment, target_rep_length)
        if np.isnan(time_norm_rep_segment).all(): repetition_results.append( (i+1, "Error: Time Norm Failed", float('inf')) ); continue
        time_norm_rep_segment = np.nan_to_num(time_norm_rep_segment, nan=0.0)

        min_rep_distance, best_rep_label_for_this_rep = float('inf'), None
        for exercise_label, angle_dict in averaged_baseline_data.items():
            baseline_to_compare = angle_dict.get(comparison_angle_key)
            if baseline_to_compare is None or time_norm_rep_segment.shape != baseline_to_compare.shape: continue

            safe_baseline = np.nan_to_num(baseline_to_compare, nan=0.0)
            try: distance, _ = fastdtw(time_norm_rep_segment, safe_baseline, dist=euclidean)
            except Exception as e: print(f"W: DTW error Rep {i+1} vs {exercise_label}/{comparison_angle_key}: {e}"); continue
            if distance < min_rep_distance: min_rep_distance, best_rep_label_for_this_rep = distance, exercise_label

        matched_label_for_rep = "No Match / Inconclusive"
        threshold_to_use = fallback_threshold
        if best_rep_label_for_this_rep:
            threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get(comparison_angle_key)
            if threshold_specific is None and comparison_angle_key != 'overall': 
                 threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get('overall')

            if threshold_specific is not None: 
                 threshold_to_use = threshold_specific
            if min_rep_distance < threshold_to_use:
                 matched_label_for_rep = best_rep_label_for_this_rep
            else: 
                 matched_label_for_rep = f"No Match (Closest: {best_rep_label_for_this_rep}, Dist: {min_rep_distance:.2f} >= {threshold_to_use:.2f})"

        repetition_results.append( (i+1, matched_label_for_rep, min_rep_distance) )

    return repetition_results, num_detected_reps

### Load baseline data

**Purpose:**

This cell executes the baseline creation and processing pipeline

**Outcomes:**

Running this cell populates the three key dictionaries that store the results of the baseline processing:

1.  `averaged_baselines`: ( `dict`)
    * **Structure:** `{exercise_label: {angle_key: numpy.array}}` (e.g., `{'squat': {'front': array([...]), 'overall': array([...])}}`)
    * **Meaning:** Contains the final averaged, time-normalized pose sequence for each detected angle ('left', 'right', 'front', 'unknown') and the 'overall' average for each exercise. These arrays represent the "template" movements used for comparison. Returns an empty dictionary if processing fails entirely.
    * Unit = Arrays contain unitless normalized coordinates.
2.  `tuned_params`: ( `dict`)
    * **Structure:** `{exercise_label: {angle_key: (prominence, distance)}}` (e.g., `{'squat': {'front': (0.06, 15), 'overall': (0.05, 10)}}`)
    * **Meaning:** Stores the optimal `(peak_prominence, peak_distance)` tuple found by the `tune_counting_parameters` function for each exercise and angle category based on the labeled baseline videos. Contains default parameters if tuning failed or no labeled videos were found for a specific category. Returns an empty dictionary if processing fails entirely.
    * Unit = Prominence (float) is unitless ratio, Distance (int) is in frames.
3.  `derived_thresholds`: ( `dict`)
    * **Structure:** `{exercise_label: {angle_key: threshold_value or None}}` (e.g., `{'squat': {'front': 450.7, 'overall': 510.2}}`)
    * **Meaning:** Stores the suggested DTW recognition threshold calculated for each exercise and angle category (including 'overall') based on the variance within its baseline repetitions (e.g., Mean + N * StdDev of intra-exercise distances). Contains `None` if a threshold couldn't be derived (e.g., too few repetitions). Returns an empty dictionary if processing fails entirely.
    * Unit = Float, unitless distance (same scale as DTW results).

**Printed Output:**

* Extensive logs generated *during* the execution of `load_baseline_data` (showing progress per exercise, tuning results, derived thresholds, etc.).
* A final summary message indicating whether the baseline data loading/processing was considered successful (based on `averaged_baselines` being populated).
* A printout of the `derived_thresholds` dictionary (or a message if it's empty/missing), allowing the user to inspect the suggested thresholds.

**Calculated Means:**

This cell triggers the execution of the entire complex pipeline defined in `load_baseline_data`. The "meaning" is the generation and storage of the structured baseline information (`averaged_baselines`, `tuned_params`, `derived_thresholds`) which encapsulates the learned characteristics (average movement, optimal counting parameters, typical variation) for each exercise and angle, ready to be used for recognition in the next cell.

In [234]:
# In case of re-running the cell
averaged_baselines = {}
tuned_params = {}
derived_thresholds = {}

# Running
averaged_baselines, tuned_params, derived_thresholds = load_baseline_data(
    BASELINE_ROOT_DIR,
    HRNET_MODEL,
    JOINT_NAMES_LIST,
    TARGET_REP_LENGTH,
    EXERCISE_CONFIG,
    TUNING_PROMINENCE_RANGE,
    TUNING_DISTANCE_RANGE,
    threshold_std_multiplier=DEFAULT_THRESHOLD_STD_MULTIPLIER,
    min_reps_for_threshold=DEFAULT_MIN_REPS_FOR_THRESHOLD
)

if averaged_baselines:
    print("\n Baseline Data Loaded/Processed Successfully👍")
    print("\n Derived recognition thresholds (Per Angle and Overall)")
else:
    print("\n Baseline Data Loading/Processing Failed👎")

Loading Baselines (Config-Driven, Angle from Filename)... from: C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/baselines2

Processing baseline exercise: tricep_pushdown (from folder: tricep Pushdown)
  Found 4 video file(s).
  Tuning repetition counting parameters for: tricep_pushdown...
Tuning complete. Best Params: P=0.020, D=30, Min Error=1. (Default Error=8)
Detected reps using BEST params (P=0.020, D=30):
Video Index 2: Detected=4, True=4
Video Index 3: Detected=4, True=5
Total Reps: Detected=8, True=9, Overall Error=1
  Tuning repetition counting parameters for: tricep_pushdown...
Tuning complete. Best Params: P=0.020, D=35, Min Error=1. (Default Error=3)
Detected reps using BEST params (P=0.020, D=35):
Video Index 0: Detected=3, True=3
Video Index 1: Detected=3, True=4
Total Reps: Detected=6, True=7, Overall Error=1

Processing baseline exercise: tricep_dips (from folder: tricep dips)
  Found 3 video file(s).

Baseline process

### Testvideo

**Purpose:**

This cell analyzing a new video file.

**User Inputs in this Cell:**

* `video_to_test`: ( `str` or `pathlib.Path`)
    * **Meaning:** Specifies the **full path** to the video file you want to classify. You should **modify this variable** each time you want to test a different video without re-running previous cells (especially Cell 6). It defaults to `VIDEO_TO_RECOGNIZE_PATH` from Cell 2 if not changed here.
* `manual_angle_hint`: ( `str` or `None`)
    * **Meaning:** Allows you to *optionally* tell the recognition function the expected camera angle ('left', 'right', 'front'). If set to one of these strings, the function will prioritize comparing the test video's repetitions against the baselines and thresholds specifically created for that angle. If left as `None`, the function will use the 'overall' baselines and thresholds (averaged across all angles).
    * Unit = Category label (`'left'`, `'right'`, `'front'`) or `None`.

**Outcomes:**

This cell does not assign results to Python variables but produces **Printed Output** to the notebook interface, summarizing the recognition attempt:

* **Header:** Indicates which video is being processed and if an angle hint was provided.
* **Detected Repetitions:** The total number of potential repetitions identified by the segmentation step.
    *  `int`, Unit = Count.
* **Per-Repetition Analysis:** A detailed list showing the result for each detected repetition:
    * Repetition Index: ( `int`).
    * Classification Status: ( `str`) - Indicates either:
        * A successful match (e.g., `-> Match: squat (Dist: 350.25)`).
        * A failure to meet the threshold (e.g., `-> No Match (Closest: squat, Dist: 600.50 >= 550.00)`).
        * An error during processing for that rep (e.g., `-> Error: Time Norm Failed`).
    * Minimum DTW Distance: ( `float`, Unit = Unitless Distance) - The lowest DTW distance found between this repetition and the relevant baseline it was compared against.
* **Overall Summary:**
    * Predominant Match: ( `str`) - The exercise label that was successfully matched most often across all repetitions (majority vote).
    * Match Counts: ( `int`/`int`, Unit = Count) - Shows how many reps were successfully matched compared to the total number detected by segmentation.

**Calculated Means / Logic:**

* **Pre-run Checks:** Verifies that all necessary variables (`averaged_baselines`, `tuned_params`, `derived_thresholds`, `HRNET_MODEL`, `exercise_config`, constants) are defined and that the `video_to_test` file exists before proceeding. Prevents errors if previous cells failed or weren't run.
* **Function Call:** Executes the main `recognize_exercise` function, passing all the required data structures generated by `load_baseline_data`, the configuration, and the user inputs (`video_to_test`, `manual_angle_hint`).
* **Result Interpretation:** Processes the list returned by `recognize_exercise` to count successful matches, determine the most frequent match (majority vote), and format the output clearly for the user.

In [235]:
video_to_test = VIDEO_TO_RECOGNIZE_PATH

# Optional angle hint
manual_angle_hint = None #'left', 'right', 'front'

if proceed_to_recognition:
    print("==========================================================================================================================================")
    print(f" Recognizing Exercise in Video")
    print(f" Video: {os.path.basename(video_to_test)} ")
    print("==========================================================================================================================================")

    list_of_rep_results, total_reps_detected = recognize_exercise(
        new_video_path=video_to_test,
        averaged_baseline_data=averaged_baselines,
        hrnet_model=HRNET_MODEL,
        joint_names=JOINT_NAMES_LIST,
        exercise_config=EXERCISE_CONFIG,
        target_rep_length=TARGET_REP_LENGTH,
        tuned_counting_params=tuned_params,
        exercise_specific_thresholds=derived_thresholds,
        hint_angle=manual_angle_hint,
        fallback_threshold=DTW_DISTANCE_THRESHOLD)

    print("\n==========================================================================================================================================")
    print(f" Results for {os.path.basename(video_to_test)}")
    print("==========================================================================================================================================")
    print(f" Detected {total_reps_detected} Repetitions.")

    if not list_of_rep_results:
        print("No valid repetition results to display")
    else:
        print("\n Per repetition Analysis")
        match_counts = {}
        successful_reps = 0
        overall_best_label = None
        valid_matches = [res[1] for res in list_of_rep_results if isinstance(res[1], str) and not res[1].startswith("Error") and not res[1].startswith("No Match")]
        if valid_matches:
             temp_match_counts = {}
             for label in valid_matches: temp_match_counts[label] = temp_match_counts.get(label, 0) + 1
             if temp_match_counts: overall_best_label = max(temp_match_counts, key=temp_match_counts.get)

        for rep_index, matched_label, min_distance in list_of_rep_results:
            status = ""
            if isinstance(matched_label, str) and matched_label.startswith("Error"):
                 status = f"-> {matched_label}"
            elif isinstance(matched_label, str) and matched_label.startswith("No Match"):
                 status = f"-> {matched_label}"
            else:
                status = f"-> Match: {matched_label} (Dist: {min_distance:.2f})"
                match_counts[matched_label] = match_counts.get(matched_label, 0) + 1
                successful_reps += 1
            print(f"    Rep {rep_index}: {status}")

        print("\n Overall Summary")
        if successful_reps > 0:
             if match_counts:
                 majority_label = max(match_counts, key=match_counts.get)
                 majority_count = match_counts[majority_label]
                 print(f"Overall match: {majority_label} ({majority_count}/{successful_reps} successfully matched reps)")
             else:
                  print(" No specific exercises were matched consistently.")
             print(f" Total reps successfully matched: {successful_reps} / {total_reps_detected}")
        elif total_reps_detected > 0:
             print("No repetitions were successfully matched to any baseline exercise.")
        else:
             print("No repetitions were detected in the video")
    print("==========================================================================================================================================\n")

 Recognizing Exercise in Video
 Video: test_excercise.mp4 

--- Recognizing Exercise (Angle Hint='None') for: test_excercise.mp4 ---
  Detected 9 potential repetitions in test video.
  Comparing each detected repetition against 'overall' baselines...

 Results for test_excercise.mp4
 Detected 9 Repetitions.

 Per repetition Analysis ---
    Rep 1: -> Match: tricep_pushdown (Dist: 1966.02)
    Rep 2: -> Match: tricep_pushdown (Dist: 1803.06)
    Rep 3: -> Match: tricep_pushdown (Dist: 1765.82)
    Rep 4: -> Match: tricep_pushdown (Dist: 2251.53)
    Rep 5: -> Match: tricep_pushdown (Dist: 2018.40)
    Rep 6: -> Match: tricep_pushdown (Dist: 2566.13)
    Rep 7: -> Match: tricep_pushdown (Dist: 2169.83)
    Rep 8: -> Match: tricep_pushdown (Dist: 1948.93)
    Rep 9: -> Match: tricep_pushdown (Dist: 1990.43)

 Overall Summary 
  Overall Predominant Match: tricep_pushdown (9/9 successfully matched reps)
 Total Reps Successfully Matched: 9 / 9



In [236]:
# === Cell 5.0 (or 5.1): Angle Detection Function (with Logic) ===
import numpy as np # Ensure numpy is imported

def detect_camera_angle(pose_sequence, joint_names, min_valid_frames=10, width_threshold_factor=0.35):
    """
    Analyzes pose sequence to determine camera angle (left, right, front)
    based on average shoulder horizontal positioning.

    Args:
        pose_sequence (np.array): Spatially normalized pose sequence (frames, coords).
                                    Should handle potential NaNs.
        joint_names (list): List of joint names.
        min_valid_frames (int): Minimum number of frames where both shoulders must be detected.
        width_threshold_factor (float): Percentage of average shoulder width used as a threshold
                                         to determine side view (e.g., 0.35 = 35%). Needs tuning.

    Returns:
        str: 'left', 'right', 'front', or 'unknown'.
    """
    if pose_sequence is None or pose_sequence.shape[0] < min_valid_frames:
        # print("DEBUG Angle: Sequence too short.") # Optional Debug
        return 'unknown' # Not enough data

    try:
        # Get indices for shoulders (ensure they exist in your JOINT_NAMES_LIST)
        lshoulder_idx = joint_names.index('Left Shoulder')
        rshoulder_idx = joint_names.index('Right Shoulder')

        # Get X coordinates (indices are 2*joint_idx + 0)
        lshoulder_x_idx = 2 * lshoulder_idx
        rshoulder_x_idx = 2 * rshoulder_idx

        # Filter frames where *both* shoulders are detected (not NaN)
        valid_frames_mask = ~np.isnan(pose_sequence[:, lshoulder_x_idx]) & \
                            ~np.isnan(pose_sequence[:, rshoulder_x_idx])

        if np.sum(valid_frames_mask) < min_valid_frames:
            # print(f"DEBUG Angle: Not enough valid shoulder frames found ({np.sum(valid_frames_mask)} < {min_valid_frames}).") # Optional Debug
            return 'unknown' # Not enough reliable data

        valid_lshoulder_x = pose_sequence[valid_frames_mask, lshoulder_x_idx]
        valid_rshoulder_x = pose_sequence[valid_frames_mask, rshoulder_x_idx]

        # Calculate average X positions
        avg_x_left = np.mean(valid_lshoulder_x)
        avg_x_right = np.mean(valid_rshoulder_x)

        # Calculate average horizontal distance between shoulders (our reference width)
        avg_shoulder_width = np.mean(np.abs(valid_lshoulder_x - valid_rshoulder_x))

        if avg_shoulder_width < 1e-6: # Avoid division by zero / handle overlapping points
             # print("DEBUG Angle: Average shoulder width near zero.") # Optional Debug
             return 'front' # Or 'unknown', front seems safer if points overlap

        # Calculate the difference (Right X - Left X)
        # Note: In standard image coordinates (origin top-left), smaller X is to the left.
        # Adjust if your normalization flips axes.
        x_difference = avg_x_right - avg_x_left

        # Determine view based on the difference relative to the average width
        threshold = avg_shoulder_width * width_threshold_factor

        detected_angle = 'front' # Default assumption
        if x_difference > threshold:
            # Right shoulder X is significantly larger than Left shoulder X
            # -> We are likely seeing the person's left side more prominently
            # -> Camera is likely positioned to the person's RIGHT
            detected_angle = 'right'
        elif x_difference < -threshold:
            # Left shoulder X is significantly larger than Right shoulder X
            # -> We are likely seeing the person's right side more prominently
            # -> Camera is likely positioned to the person's LEFT
            detected_angle = 'left'
        # Else: Shoulders are relatively aligned horizontally -> assume front view

        # print(f"DEBUG Angle: AvgL={avg_x_left:.2f}, AvgR={avg_x_right:.2f}, Diff={x_difference:.2f}, Width={avg_shoulder_width:.2f}, Thresh={threshold:.2f} -> Angle={detected_angle}") # Optional Debug

        return detected_angle

    except (ValueError, IndexError) as e:
        print(f"Warning: Could not find required joints ('Left Shoulder', 'Right Shoulder') for angle detection: {e}")
        return 'unknown'
    except Exception as e:
        print(f"Warning: Error during angle detection: {e}")
        return 'unknown'

print("Angle detection function defined (detect_camera_angle) with shoulder logic.")

Angle detection function defined (detect_camera_angle) with shoulder logic.


In [237]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

HRNET_MODEL = None
try:
    HRNET_MODEL = SimpleHRNet(c=48, 
                              nof_joints=EXPECTED_JOINT_COUNT,
                              checkpoint_path=MODEL_PATH,
                              device=DEVICE,
                              multiperson=False, # Logic currently assumes single person
                              max_batch_size=16) # Added default batch size
    print("SimpleHRNet model loaded successfully.")
except NameError:
    print("Error: SimpleHRNet class not found. Check import in Cell 1.")
except FileNotFoundError as fnf_error:
    print(fnf_error) # Print the specific error
except Exception as e:
    print(f"Error loading SimpleHRNet model: {e}")

Using device: cpu
device: 'cpu'
SimpleHRNet model loaded successfully.


In [238]:
def get_keypoint(frame_kps, kp_index):
    """Safely get keypoint coordinates (x, y) from a frame's keypoint array."""
    if frame_kps is not None and kp_index < frame_kps.shape[0]:
        coords = frame_kps[kp_index, :2] # Take only x, y
        if not np.isnan(coords).any():
            return coords
    return None

def calculate_angle(p1, p2, p3):
    """Calculates the angle (in degrees) at p2 formed by p1p2p3."""
    if p1 is None or p2 is None or p3 is None: return None
    v1, v2 = np.array(p1) - np.array(p2), np.array(p3) - np.array(p2)
    mag1, mag2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if mag1 * mag2 == 0: return None
    cos_angle = np.clip(np.dot(v1, v2) / (mag1 * mag2), 1.0, 1.0)
    return np.degrees(np.arccos(cos_angle))

def handle_nan_values(sequence):
    """Handles NaN values in a pose sequence (frames, coords). Removes frames containing any NaNs."""
    if sequence is None or sequence.ndim != 2 or sequence.shape[0] == 0:
        return np.empty((0, sequence.shape[1] if sequence is not None and sequence.ndim == 2 else 0))
    valid_frames_mask = ~np.isnan(sequence).any(axis=1)
    return sequence[valid_frames_mask]

def time_normalize_sequence(sequence, target_length):
    """Resamples a pose sequence (frames, coords) to a target length using linear interpolation."""
    if sequence is None or sequence.shape[0] < 2:
        num_coords = sequence.shape[1] if sequence is not None and sequence.ndim == 2 else EXPECTED_JOINT_COUNT * 2
        return np.full((target_length, num_coords), np.nan)

    num_frames, num_coords = sequence.shape
    original_indices = np.linspace(0, num_frames - 1, num_frames)
    target_indices = np.linspace(0, num_frames - 1, target_length)
    normalized_sequence = np.zeros((target_length, num_coords))

    for j in range(num_coords):
        valid_mask = ~np.isnan(sequence[:, j])
        if np.sum(valid_mask) < 2:
            normalized_sequence[:, j] = np.nan
        else:
            try:
                normalized_sequence[:, j] = np.interp(target_indices, original_indices[valid_mask], sequence[valid_mask, j])
            except Exception:
                normalized_sequence[:, j] = np.nan

    nan_mask = np.isnan(normalized_sequence)
    if np.any(nan_mask):
        idx = np.where(~nan_mask, np.arange(nan_mask.shape[0])[:, None], 0)
        np.maximum.accumulate(idx, axis=0, out=idx)
        normalized_sequence[nan_mask] = normalized_sequence[idx[nan_mask], np.nonzero(nan_mask)[1]]
        nan_mask = np.isnan(normalized_sequence)
        if np.any(nan_mask):
            idx = np.where(~nan_mask, np.arange(nan_mask.shape[0])[:, None], nan_mask.shape[0] - 1)
            idx = np.minimum.accumulate(idx[::1], axis=0)[::1]
            normalized_sequence[nan_mask] = normalized_sequence[idx[nan_mask], np.nonzero(nan_mask)[1]]

    return np.nan_to_num(normalized_sequence, nan=0.0)

print("Helper functions defined.")

Helper functions defined.


In [239]:
# === Cell 5.0: Angle Detection & Imports for Core Functions ===

import numpy as np
import os
import re # For parsing filenames
import time
import math
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.signal import find_peaks
import scipy.ndimage # For smoothing (alternative)
import pandas as pd # For robust NaN filling

# Make sure constants from Cell 2 are accessible (e.g., JOINT_NAMES_LIST)
# Make sure helpers from Cell 4 are accessible (get_keypoint, calculate_angle)

def detect_camera_angle(pose_sequence, joint_names, min_valid_frames=10, width_threshold_factor=0.35):
    """
    Analyzes pose sequence for camera angle based on shoulder positions.
    Placeholder heuristic - needs validation and likely improvement.
    """
    if pose_sequence is None or pose_sequence.shape[0] < min_valid_frames: return 'unknown'
    try:
        lshoulder_idx = joint_names.index('Left Shoulder')
        rshoulder_idx = joint_names.index('Right Shoulder')
        lshoulder_x_idx, rshoulder_x_idx = 2 * lshoulder_idx, 2 * rshoulder_idx
        # Ensure indices are valid before accessing pose_sequence columns
        if max(lshoulder_x_idx, rshoulder_x_idx) >= pose_sequence.shape[1]:
             # print("W: Shoulder indices out of bounds for pose_sequence in angle detection.")
             return 'unknown'

        valid_mask = ~np.isnan(pose_sequence[:, lshoulder_x_idx]) & ~np.isnan(pose_sequence[:, rshoulder_x_idx])
        if np.sum(valid_mask) < min_valid_frames: return 'unknown'

        valid_lsx = pose_sequence[valid_mask, lshoulder_x_idx]
        valid_rsx = pose_sequence[valid_mask, rshoulder_x_idx]
        avg_x_left, avg_x_right = np.mean(valid_lsx), np.mean(valid_rsx)
        avg_shoulder_width = np.mean(np.abs(valid_lsx - valid_rsx))

        if avg_shoulder_width < 1e-6: return 'front'
        x_difference = avg_x_right - avg_x_left
        threshold = avg_shoulder_width * width_threshold_factor

        if x_difference > threshold: return 'right' # Seeing left side -> Camera on right
        elif x_difference < -threshold: return 'left' # Seeing right side -> Camera on left
        else: return 'front'
    except (ValueError, IndexError): # Catch if joints aren't in list
        return 'unknown'
    except Exception as e:
        return 'unknown'

print("Angle detection function defined (detect_camera_angle). Imports for Cell 5 loaded.")

Angle detection function defined (detect_camera_angle). Imports for Cell 5 loaded.


In [240]:
### Cell 5.1: Basic Helper Functions

In [241]:
# === Cell 5.1: Video Processing & Normalization Functions ===
# Requires numpy, os, cv2, get_keypoint
# Assumes EXPECTED_JOINT_COUNT, JOINT_NAMES_LIST are defined

def process_single_video(video_path, model, expected_joint_count=EXPECTED_JOINT_COUNT):
    """Processes a single video, extracts joints, returns NumPy array (frames, num_coords)."""
    if not os.path.exists(video_path): print(f"E: Video not found: {video_path}"); return None
    video = cv2.VideoCapture(video_path);
    if not video.isOpened(): print(f"E: Cannot open video: {video_path}"); return None
    video_joints, frame_count = [], 0; expected_coords = expected_joint_count * 2
    try:
        while True:
            ret, frame = video.read();
            if not ret: break
            try:
                joints = model.predict(frame)
                if joints is None or joints.shape[0]==0 or joints.shape[1]!=expected_joint_count: frame_coords=np.full(expected_coords,np.nan)
                else: frame_coords=joints[0,:,:2].flatten(); frame_coords = frame_coords if frame_coords.shape[0]==expected_coords else np.full(expected_coords,np.nan)
            except Exception as e: frame_coords = np.full(expected_coords, np.nan)
            video_joints.append(frame_coords); frame_count += 1
    finally: video.release()
    return np.array(video_joints) if video_joints else None

def normalize_pose_sequence(pose_sequence, joint_names, ref_joint1_name="Left Shoulder", ref_joint2_name="Right Shoulder"):
    """Spatially normalizes a pose sequence based on reference joints."""
    if pose_sequence is None or pose_sequence.shape[0] == 0: return np.array([])
    try: ref_joint1_idx, ref_joint2_idx = joint_names.index(ref_joint1_name), joint_names.index(ref_joint2_name)
    except ValueError: print(f"E: Ref joints not found in normalize"); return np.full_like(pose_sequence, np.nan)
    normalized_frames = []; num_coords = pose_sequence.shape[1]
    if num_coords == 0 or num_coords % 2 != 0 or num_coords != len(joint_names) * 2:
        print(f"W: Coord mismatch/zero coords in normalize_pose_sequence ({num_coords} vs {len(joint_names)*2}). Returning NaNs.")
        return np.full_like(pose_sequence, np.nan)

    for frame in pose_sequence:
        try:
             if frame.shape[0] != num_coords: normalized_frames.append(np.full(num_coords, np.nan)); continue
             frame_joints = frame.reshape(-1, 2);
             if frame_joints.shape[0] != len(joint_names): normalized_frames.append(np.full(num_coords, np.nan)); continue
             ref1, ref2 = get_keypoint(frame_joints, ref_joint1_idx), get_keypoint(frame_joints, ref_joint2_idx)
             if ref1 is None or ref2 is None: normalized_frames.append(np.full(num_coords, np.nan)); continue
             center, scale = (np.array(ref1) + np.array(ref2)) / 2.0, np.linalg.norm(np.array(ref1) - np.array(ref2))
             if scale == 0 or np.isclose(scale, 0) or np.isnan(scale): normalized_frames.append(np.full(num_coords, np.nan)); continue
             normalized_frames.append(((frame_joints - center) / scale).flatten())
        except Exception: normalized_frames.append(np.full(num_coords, np.nan)); continue
    return np.array(normalized_frames)

print("Video processing and normalization functions defined.")

Video processing and normalization functions defined.


In [242]:
# === Cell 5.2: Video Processing & Normalization Functions ===

In [243]:
# === Cell 5.2: Trajectory Calculation Helper ===
# Requires numpy, get_keypoint, calculate_angle
# Assumes JOINT_NAMES_LIST is defined

def _calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_config):
    """Calculates the 1D trajectory signal based on configuration."""
    if pose_sequence is None or pose_sequence.shape[0] < 2: return None, False

    metric = trajectory_config.get('metric', 'unknown')
    base_joint_names = trajectory_config.get('joints', [])
    invert_for_valley = trajectory_config.get('invert_for_valley', False)
    trajectory = None

    try:
        if not base_joint_names: raise ValueError("No 'joints' specified in trajectory_config")

        # --- Logic based on metric ---
        if metric.endswith('_angle'):
            if len(base_joint_names) != 3: raise ValueError(f"{metric} needs 3 joints in config")
            j1_base, j2_base, j3_base = base_joint_names[0], base_joint_names[1], base_joint_names[2]
            angles_left, angles_right = [], []
            try: # Try finding Left/Right pairs first
                l_j1_idx, l_j2_idx, l_j3_idx = joint_names.index(f'Left {j1_base}'), joint_names.index(f'Left {j2_base}'), joint_names.index(f'Left {j3_base}')
                r_j1_idx, r_j2_idx, r_j3_idx = joint_names.index(f'Right {j1_base}'), joint_names.index(f'Right {j2_base}'), joint_names.index(f'Right {j3_base}')
                for frame_coords in pose_sequence:
                    f_j = frame_coords.reshape(-1, 2)
                    p1l,p2l,p3l = get_keypoint(f_j,l_j1_idx), get_keypoint(f_j,l_j2_idx), get_keypoint(f_j,l_j3_idx)
                    p1r,p2r,p3r = get_keypoint(f_j,r_j1_idx), get_keypoint(f_j,r_j2_idx), get_keypoint(f_j,r_j3_idx)
                    angles_left.append(calculate_angle(p1l, p2l, p3l) if all(p is not None for p in [p1l, p2l, p3l]) else np.nan)
                    angles_right.append(calculate_angle(p1r, p2r, p3r) if all(p is not None for p in [p1r, p2r, p3r]) else np.nan)
                traj_l, traj_r = np.array(angles_left), np.array(angles_right)
                trajectory = np.nanmean([traj_l, traj_r], axis=0) # Average left/right
            except ValueError: # Fallback if L/R pairing doesn't exist
                j1_idx, j2_idx, j3_idx = joint_names.index(j1_base), joint_names.index(j2_base), joint_names.index(j3_base)
                angles = []
                for frame_coords in pose_sequence:
                     f_j = frame_coords.reshape(-1, 2)
                     p1, p2, p3 = get_keypoint(f_j, j1_idx), get_keypoint(f_j, j2_idx), get_keypoint(f_j, j3_idx)
                     angles.append(calculate_angle(p1, p2, p3) if all(p is not None for p in [p1, p2, p3]) else np.nan)
                trajectory = np.array(angles)

        elif metric.endswith('_y') or metric.endswith('_x'): # Coordinate metric
            coord_idx = 1 if metric.endswith('_y') else 0 # 1 for Y, 0 for X
            base_joint = base_joint_names[0]
            try: # Try L/R average
                 l_joint_idx = joint_names.index(f'Left {base_joint}')
                 r_joint_idx = joint_names.index(f'Right {base_joint}')
                 # Check if indices are valid before accessing pose_sequence
                 if max(2*l_joint_idx+coord_idx, 2*r_joint_idx+coord_idx) >= pose_sequence.shape[1]: raise ValueError("Joint index out of bounds")
                 l_coord = pose_sequence[:, 2 * l_joint_idx + coord_idx]
                 r_coord = pose_sequence[:, 2 * r_joint_idx + coord_idx]
                 trajectory = np.nanmean([l_coord, r_coord], axis=0)
            except ValueError: # Fallback to single joint coord
                 joint_idx = joint_names.index(base_joint)
                 if 2*joint_idx+coord_idx >= pose_sequence.shape[1]: raise ValueError("Joint index out of bounds")
                 trajectory = pose_sequence[:, 2 * joint_idx + coord_idx]
        else:
             raise ValueError(f"Unknown metric type specified in config: '{metric}'")

        # --- Post-processing trajectory ---
        if trajectory is None or trajectory.shape[0] == 0 or np.all(np.isnan(trajectory)): return None, False
        nan_mask = np.isnan(trajectory)
        if np.any(nan_mask):
            indices = np.arange(len(trajectory)); valid_indices = np.flatnonzero(~nan_mask)
            if len(valid_indices) < 2: return None, False # Cannot interpolate
            trajectory[nan_mask] = np.interp(indices[nan_mask], valid_indices, trajectory[valid_indices])
        if np.any(np.isnan(trajectory)): trajectory = np.nan_to_num(trajectory, nan=np.nanmean(trajectory)) # Fill with mean as fallback

        return trajectory, invert_for_valley

    except (ValueError, IndexError) as e: print(f"W: Error finding/using joints for metric '{metric}': {e}"); return None, False
    except Exception as e: print(f"E: Error calculating trajectory with metric '{metric}': {e}"); return None, False

print("Trajectory calculation helper defined (_calculate_trajectory_from_config).")

Trajectory calculation helper defined (_calculate_trajectory_from_config).


In [244]:
# === Cell 5.3: Trajectory Calculation Function ===

In [245]:
# === Cell 5.3: Peak Finding Helper Function ===
# Requires numpy, scipy.signal.find_peaks

def _extract_trajectory_and_find_peaks(trajectory, smoothing_window, prominence, distance, invert_trajectory=False):
    """Internal helper to smooth a trajectory and find peaks."""
    if trajectory is None or trajectory.ndim != 1 or trajectory.shape[0] < max(distance * 2, smoothing_window, 1): return None # Added 1 for edge case
    processed_trajectory = trajectory.copy()
    if invert_trajectory: processed_trajectory = -processed_trajectory
    if len(processed_trajectory) < smoothing_window: return None # Cannot smooth

    # Use valid mode for convolution, check length after smoothing
    smoothed_trajectory = np.convolve(processed_trajectory, np.ones(smoothing_window)/smoothing_window, mode='valid')
    if smoothed_trajectory.shape[0] < max(distance, 1): return None # Need enough points for distance check in find_peaks

    data_range = np.ptp(smoothed_trajectory)
    required_prominence = max(data_range * prominence, 1e-6) if data_range > 1e-9 else 1e-6

    try:
        # Calculate offset correctly for 'valid' mode convolution
        smoothed_indices_offset = (len(processed_trajectory) - len(smoothed_trajectory)) // 2
        peak_indices_smoothed, _ = find_peaks(smoothed_trajectory, prominence=required_prominence, distance=distance)
        peak_indices_original = peak_indices_smoothed + smoothed_indices_offset
        # Ensure indices are within bounds of original trajectory
        peak_indices_original = peak_indices_original[(peak_indices_original >= 0) & (peak_indices_original < len(processed_trajectory))]
        return peak_indices_original
    except Exception as e: print(f"W: Peak finding failed: {e}"); return None

print("Peak finding helper function defined (_extract_trajectory_and_find_peaks).")

Peak finding helper function defined (_extract_trajectory_and_find_peaks).


In [246]:
# === Cell 5.4: Peak Finding Helper Function ===

In [247]:
# === Cell 5.4: Segmentation & Counting Functions ===
# Requires _calculate_trajectory_from_config, _extract_trajectory_and_find_peaks
# Assumes constants like DEFAULT_SMOOTHING_WINDOW etc. are defined

def segment_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, # Pass full config dict
                        smoothing_window=DEFAULT_SMOOTHING_WINDOW,
                        peak_prominence=DEFAULT_PEAK_PROMINENCE,
                        peak_distance=DEFAULT_PEAK_DISTANCE):
    """Segments a pose sequence into individual repetitions using EXERCISE_CONFIG."""
    # Get specific config for this exercise
    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    # If no specific config, maybe try a default or skip? For now, skip.
    if not current_exercise_cfg_dict:
        # print(f"W: No config found for {exercise_label} in segment_repetitions") # Optional Warning
        return []
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg:
         # print(f"W: No trajectory_logic found in config for {exercise_label}") # Optional Warning
         return []

    # 1. Calculate trajectory based on config
    trajectory, invert = _calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_logic_cfg)

    # 2. Find peaks using specified or default parameters
    peak_indices = _extract_trajectory_and_find_peaks(trajectory, smoothing_window, peak_prominence, peak_distance, invert_trajectory=invert)

    if peak_indices is None or len(peak_indices) < 2: return []

    # 3. Segment the original pose_sequence based on peak indices
    repetitions = [pose_sequence[peak_indices[i]:peak_indices[i+1], :] for i in range(len(peak_indices) - 1)]
    return [rep for rep in repetitions if rep.shape[0] > 1] # Filter out tiny segments

def count_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, # Pass full config dict
                      prominence=DEFAULT_PEAK_PROMINENCE, distance=DEFAULT_PEAK_DISTANCE, # Use potentially tuned params
                      smoothing_window=DEFAULT_SMOOTHING_WINDOW):
    """Counts repetitions using EXERCISE_CONFIG and specified parameters."""
    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    if not current_exercise_cfg_dict: return 0 # Cannot count without config
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg: return 0 # Cannot count without trajectory logic

    # 1. Calculate trajectory based on config
    trajectory, invert = _calculate_trajectory_from_config(pose_sequence, joint_names, trajectory_logic_cfg)

    # 2. Find peaks using specified parameters
    peak_indices = _extract_trajectory_and_find_peaks(trajectory, smoothing_window, prominence, distance, invert_trajectory=invert)

    return len(peak_indices) if peak_indices is not None else 0

print("Segmentation and counting functions defined (using EXERCISE_CONFIG).")

Segmentation and counting functions defined (using EXERCISE_CONFIG).


In [248]:
# === Cell 5.5: Segmentation & Counting Functions ===

In [249]:
# === Cell 5.5: Parameter Tuning Function ===
# Requires _calculate_trajectory_from_config, _extract_trajectory_and_find_peaks
# Assumes constants like DEFAULT_PEAK_PROMINENCE etc. are defined

def tune_counting_parameters(exercise_label, labeled_video_data, joint_names, exercise_config, # Pass full config dict
                             prominence_range, distance_range):
    """Performs grid search using the refactored peak finding and config."""
    print(f"  Tuning repetition counting parameters for: {exercise_label}...")
    best_params = (DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE)
    min_total_error = float('inf')
    if not labeled_video_data: print("W: No labeled data for tuning."); return best_params

    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    if not current_exercise_cfg_dict: print(f"W: No config for {exercise_label} in tuning"); return best_params
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg: print(f"W: No trajectory_logic in config for {exercise_label}"); return best_params

    num_combinations = len(prominence_range) * len(distance_range)
    print(f"    Testing {num_combinations} parameter combinations...")
    default_error = 0
    precalculated_trajectories = {}

    # Pre-calculate trajectories
    for i, (vid_idx, (sequence, true_reps)) in enumerate(labeled_video_data): # Expecting (original_video_idx, (sequence, true_reps))
        trajectory, invert = _calculate_trajectory_from_config(sequence, joint_names, trajectory_logic_cfg)
        if trajectory is not None: precalculated_trajectories[vid_idx] = (trajectory, invert, true_reps) # Use video_idx as key

    if not precalculated_trajectories: print("W: No valid trajectories for tuning."); return best_params

    # Calculate default error
    for vid_idx, (traj, inv, true_r) in precalculated_trajectories.items():
         peak_indices_def = _extract_trajectory_and_find_peaks(traj, DEFAULT_SMOOTHING_WINDOW, DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE, invert_trajectory=inv)
         calc_reps_def = len(peak_indices_def) if peak_indices_def is not None else 0
         default_error += abs(calc_reps_def - true_r)

    # Grid search
    for p_idx, p in enumerate(prominence_range):
        for d_idx, d in enumerate(distance_range):
            current_total_error = 0
            for vid_idx, (traj, inv, true_r) in precalculated_trajectories.items():
                peak_indices = _extract_trajectory_and_find_peaks(traj, DEFAULT_SMOOTHING_WINDOW, p, d, invert_trajectory=inv)
                calculated_reps = len(peak_indices) if peak_indices is not None else 0
                current_total_error += abs(calculated_reps - true_r)
            if current_total_error < min_total_error:
                min_total_error = current_total_error
                best_params = (p, d)
            # Stop early if perfect match found (error=0)
            if current_total_error == 0: break
        if current_total_error == 0: break


    print(f"    Tuning complete. Best Params: P={best_params[0]:.3f}, D={best_params[1]}, Min Error={min_total_error}. (Default Error={default_error})")

    # Print detected reps with best params
    if precalculated_trajectories:
        print(f"    Detected reps using BEST params (P={best_params[0]:.3f}, D={best_params[1]}):")
        total_calculated, total_true = 0, 0
        # Sort by video index for consistent output
        for vid_idx, (traj, inv, true_r) in sorted(precalculated_trajectories.items()):
            peak_indices = _extract_trajectory_and_find_peaks(traj, DEFAULT_SMOOTHING_WINDOW, best_params[0], best_params[1], invert_trajectory=inv)
            calculated_reps = len(peak_indices) if peak_indices is not None else 0
            print(f"      - Video Index {vid_idx}: Detected={calculated_reps}, True={true_r}")
            total_calculated += calculated_reps; total_true += true_r
        print(f"    -------------------------------------")
        print(f"    Total Reps: Detected={total_calculated}, True={total_true}, Overall Error={abs(total_calculated - total_true)}")

    return best_params

print("Parameter tuning function defined (tune_counting_parameters).")

Parameter tuning function defined (tune_counting_parameters).


In [250]:
# === Cell 5.6: Parameter Tuning Function ===

In [251]:
# === Cell 5.6: Baseline Loading Function ===
# Requires all helpers, processing functions, tuning function defined above
# Assumes constants like BASELINE_ROOT_DIR etc. are defined

def load_baseline_data(baseline_root_dir, hrnet_model, joint_names, target_rep_length, exercise_config, # Pass config
                       prominence_range, distance_range, # Pass tuning ranges
                       threshold_std_multiplier=DEFAULT_THRESHOLD_STD_MULTIPLIER,
                       min_reps_for_threshold=DEFAULT_MIN_REPS_FOR_THRESHOLD):
    """
    Loads baselines, uses EXERCISE_CONFIG, parses angle from filename,
    tunes params per angle, averages per angle AND overall, calculates thresholds.
    Returns nested dicts: averaged_baselines, tuned_counting_params, exercise_specific_thresholds
    """
    averaged_baseline_data = {} # Structure: {exercise: {angle: avg_sequence}}
    tuned_counting_params = {}  # Structure: {exercise: {angle: (p, d)}}
    exercise_specific_thresholds = {} # Structure: {exercise: {angle: threshold}}
    start_time_total = time.time()

    print(f"Loading Baselines (Config-Driven, Angle from Filename)... from: {baseline_root_dir}")
    if not os.path.isdir(baseline_root_dir): return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds

    for exercise_label_raw in sorted(os.listdir(baseline_root_dir)):
        exercise_folder_path = os.path.join(baseline_root_dir, exercise_label_raw)
        if not os.path.isdir(exercise_folder_path): continue
        exercise_label = exercise_label_raw.lower().replace(" ", "_")
        print(f"\nProcessing baseline exercise: {exercise_label} (from folder: {exercise_label_raw})")

        current_exercise_config_dict = exercise_config.get(exercise_label)
        if not current_exercise_config_dict:
             print(f"  W: No config found in EXERCISE_CONFIG for '{exercise_label}'. Skipping this exercise.")
             continue # Skip to the next exercise if no config defined

        all_reps_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []}
        # Store as {angle: [(vid_idx, (sequence, true_reps)), ...]}
        labeled_data_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []}
        averaged_baseline_data[exercise_label] = {}
        tuned_counting_params[exercise_label] = {}
        exercise_specific_thresholds[exercise_label] = {}
        video_files = sorted([f for f in os.listdir(exercise_folder_path) if f.lower().endswith(('.mp4', '.avi', '.mov'))])
        print(f"  Found {len(video_files)} video file(s).")

        # Loop 1: Process videos, parse angle/reps, segment, normalize time, store reps by PARSED angle
        for video_idx, video_file in enumerate(video_files):
            video_path = os.path.join(exercise_folder_path, video_file)
            true_reps, parsed_angle = None, 'unknown'
            rep_match = re.search(r'_(\d+)reps', video_file, re.IGNORECASE)
            if rep_match: true_reps = int(rep_match.group(1))
            angle_match = re.search(r'_(left|right|front)', video_file, re.IGNORECASE)
            if angle_match: parsed_angle = angle_match.group(1).lower()
            # print(f"    Video {video_idx+1}: {video_file}, Reps={true_reps}, Angle={parsed_angle}") # Verbose

            raw_seq = process_single_video(video_path, hrnet_model)
            if raw_seq is None or raw_seq.shape[0] == 0: continue
            spatially_norm_seq = normalize_pose_sequence(raw_seq, joint_names)
            if spatially_norm_seq is None or np.all(np.isnan(spatially_norm_seq)): continue
            cleaned_seq = handle_nan_values(spatially_norm_seq)
            if cleaned_seq.shape[0] < 2: continue
            if true_reps is not None:
                 # Store original video index for reference in tuning output
                 labeled_data_by_angle[parsed_angle].append((video_idx, (cleaned_seq, true_reps)))
#####
            
            # Segment using default params initially for consistent rep extraction for averaging
            # Pass the specific config dict now
            repetitions = segment_repetitions(cleaned_seq, exercise_label, joint_names, current_exercise_config_dict) # ADDED argument
            if not repetitions: continue
            for rep_segment in repetitions:
                if rep_segment is not None and rep_segment.shape[0] >= 2:
                    time_norm_rep = time_normalize_sequence(rep_segment, target_rep_length)
                    # Check shape consistency (all should have target_rep_length * num_coords)
                    expected_coords = len(joint_names) * 2
                    if not np.isnan(time_norm_rep).all() and time_norm_rep.shape == (target_rep_length, expected_coords):
                        all_reps_by_angle[parsed_angle].append(time_norm_rep)
                    # else: print(f"W: Skipping rep segment due to NaN/shape mismatch after time norm. Shape: {time_norm_rep.shape}") # Verbose


        # Loop 2: Process per detected angle category
        all_reps_across_angles = []
        for angle in ['left', 'right', 'front', 'unknown']:
            # Get data for tuning, pass only (sequence, true_reps) tuples
            labeled_data_for_this_angle_tuples = [data for idx, data in labeled_data_by_angle[angle]]
            normalized_reps_for_this_angle = all_reps_by_angle[angle]
            if not normalized_reps_for_this_angle: continue # Skip if no reps for this angle
            print(f"\n  Processing angle category: '{angle}' for {exercise_label}")
            all_reps_across_angles.extend(normalized_reps_for_this_angle)

            # Tune parameters for this angle if labeled data exists
            if labeled_data_for_this_angle_tuples:
                 tuned_p, tuned_d = tune_counting_parameters(exercise_label, labeled_data_by_angle[angle], # Pass original list with indices
                                                            joint_names, exercise_config, prominence_range, distance_range)
                 tuned_counting_params[exercise_label][angle] = (tuned_p, tuned_d)
            else: tuned_counting_params[exercise_label][angle] = (DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE)

            # Average sequences for this angle
            reps_stack = np.stack(normalized_reps_for_this_angle, axis=0)
            averaged_sequence_angle = np.nanmean(reps_stack, axis=0)
            averaged_baseline_data[exercise_label][angle] = np.nan_to_num(averaged_sequence_angle, nan=0.0)
            print(f"    Stored averaged baseline for angle '{angle}'.")

            # Derive threshold for this angle
            derived_threshold_angle = None
            if len(normalized_reps_for_this_angle) >= min_reps_for_threshold:
                intra_distances_angle = []
                safe_avg_angle = np.nan_to_num(averaged_sequence_angle, nan=0.0)
                for norm_rep in normalized_reps_for_this_angle:
                     safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                     if safe_rep.shape == safe_avg_angle.shape:
                         try: dist, _ = fastdtw(safe_rep, safe_avg_angle, dist=euclidean); intra_distances_angle.append(dist)
                         except Exception: pass # Ignore DTW errors during threshold calc? Or handle?
                # Check if enough valid distances were calculated
                if len(intra_distances_angle) >= min_reps_for_threshold:
                     mean_dist, std_dist = np.mean(intra_distances_angle), np.std(intra_distances_angle)
                     derived_threshold_angle = mean_dist + threshold_std_multiplier * std_dist
                     print(f"    Derived Threshold for angle '{angle}': {derived_threshold_angle:.2f} (Mean={mean_dist:.2f}, Std={std_dist:.2f})")
                # else: print(f"W: Not enough valid distances ({len(intra_distances_angle)}) calculated for threshold angle '{angle}'.") # Verbose
            exercise_specific_thresholds[exercise_label][angle] = derived_threshold_angle

        # Calculate and Store Overall Average and Threshold
        print(f"\n  Processing 'overall' category for {exercise_label}")
        if len(all_reps_across_angles) >= min_reps_for_threshold:
             print(f"    Averaging {len(all_reps_across_angles)} total repetitions for 'overall' baseline.")
             overall_reps_stack = np.stack(all_reps_across_angles, axis=0)
             overall_averaged_sequence = np.nanmean(overall_reps_stack, axis=0)
             averaged_baseline_data[exercise_label]['overall'] = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             print(f"    Stored 'overall' averaged baseline.")

             overall_intra_distances = []
             safe_overall_avg = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             for norm_rep in all_reps_across_angles:
                  safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                  if safe_rep.shape == safe_overall_avg.shape:
                       try: dist, _ = fastdtw(safe_rep, safe_overall_avg, dist=euclidean); overall_intra_distances.append(dist)
                       except Exception: pass
             if len(overall_intra_distances) >= min_reps_for_threshold:
                  mean_dist, std_dist = np.mean(overall_intra_distances), np.std(overall_intra_distances)
                  derived_threshold_overall = mean_dist + threshold_std_multiplier * std_dist
                  exercise_specific_thresholds[exercise_label]['overall'] = derived_threshold_overall
                  print(f"    Derived 'Overall' Threshold: {derived_threshold_overall:.2f} (Mean={mean_dist:.2f}, Std={std_dist:.2f})")
             else: exercise_specific_thresholds[exercise_label]['overall'] = None
        else:
             print(f"    W: Not enough total valid reps ({len(all_reps_across_angles)}) to create 'overall' baseline/threshold.")
             averaged_baseline_data[exercise_label]['overall'] = None
             exercise_specific_thresholds[exercise_label]['overall'] = None

    end_time_total = time.time()
    print(f"\nBaseline processing complete in {end_time_total - start_time_total:.2f}s.")
    return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds


# --- Recognition Function (Using Config, Angle Hint or Overall) ---
def recognize_exercise(new_video_path, averaged_baseline_data, hrnet_model, joint_names, exercise_config, # Pass config
                       target_rep_length, tuned_counting_params, exercise_specific_thresholds,
                       hint_angle=None, fallback_threshold=DTW_DISTANCE_THRESHOLD):
    """Recognizes exercise using angle hint, config-driven logic, angle-specific baselines/thresholds."""
    print(f"\n--- Recognizing Exercise (Angle Hint='{hint_angle}') for: {os.path.basename(new_video_path)} ---")
    start_time = time.time()
    if tuned_counting_params is None: tuned_counting_params = {}
    if exercise_specific_thresholds is None: exercise_specific_thresholds = {}
    default_p, default_d = DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE

    # Step 1: Process Video -> Spatial Norm -> Clean
    new_raw_seq = process_single_video(new_video_path, hrnet_model)
    if new_raw_seq is None or new_raw_seq.shape[0] == 0: print("E: Failed processing video."); return [], 0
    new_spatially_norm_seq = normalize_pose_sequence(new_raw_seq, joint_names)
    if new_spatially_norm_seq is None or np.all(np.isnan(new_spatially_norm_seq)): print("E: Failed spatial normalization."); return [], 0
    new_sequence_clean = handle_nan_values(new_spatially_norm_seq)
    if new_sequence_clean.shape[0] < 2: print("W: Sequence too short after cleaning."); return [], 0

    # Step 2: Time-Normalize WHOLE Cleaned Sequence for Initial Guess
    new_sequence_time_norm = time_normalize_sequence(new_sequence_clean, target_rep_length)
    if np.isnan(new_sequence_time_norm).all(): print("E: Time normalization failed for whole sequence."); return [], 0
    new_sequence_time_norm = np.nan_to_num(new_sequence_time_norm, nan=0.0)

    # Step 3: Initial Guess (using 'overall' baselines) & Parameter Selection
    initial_min_distance, initial_best_label = float('inf'), None
    segment_p, segment_d = default_p, default_d
    segment_exercise_config_dict = None

    if not averaged_baseline_data: print("E: No baseline data."); return [], 0
    # print("  Performing initial comparison against 'overall' baselines...") # Verbose
    for ex_label, angle_dict in averaged_baseline_data.items():
        avg_base_seq = angle_dict.get('overall')
        if avg_base_seq is None or new_sequence_time_norm.shape != avg_base_seq.shape: continue
        safe_base = np.nan_to_num(avg_base_seq, nan=0.0)
        try: dist, _ = fastdtw(new_sequence_time_norm, safe_base, dist=euclidean)
        except Exception: continue
        if dist < initial_min_distance: initial_min_distance, initial_best_label = dist, ex_label

    # Select segmentation parameters
    angle_for_tuning_lookup = hint_angle if hint_angle in ['left', 'right', 'front'] else 'front'
    if initial_best_label:
        segment_exercise_config_dict = exercise_config.get(initial_best_label)
        if segment_exercise_config_dict:
             tuned_p, tuned_d = tuned_counting_params.get(initial_best_label, {}).get(angle_for_tuning_lookup, (default_p, default_d))
             segment_p, segment_d = tuned_p, tuned_d
             # print(f"  Initial guess: {initial_best_label}. Using its '{angle_for_tuning_lookup}' tuned params for segmentation (P={segment_p:.3f}, D={segment_d}).") # Verbose
        else: initial_best_label = None # Reset guess if no config
    # else: print(f"  Initial guess failed/inconclusive. Using default params for segmentation.") # Verbose

    # Step 4: Segment Test Video
    # Pass label guess (or unknown) and the full config dict
    segment_label_for_logic = initial_best_label if initial_best_label else "unknown"
    test_repetitions = segment_repetitions(new_sequence_clean, segment_label_for_logic, joint_names, exercise_config,
                                           peak_prominence=segment_p, peak_distance=segment_d)
    num_detected_reps = len(test_repetitions)
    print(f"  Detected {num_detected_reps} potential repetitions in test video.")
    if num_detected_reps == 0: return [], 0

    # Step 5: Compare Each Repetition Segment (Angle-Aware)
    repetition_results = []
    comparison_angle_key = hint_angle if hint_angle in ['left', 'right', 'front'] else 'overall'
    print(f"  Comparing each detected repetition against '{comparison_angle_key}' baselines...")

    for i, rep_segment in enumerate(test_repetitions):
        # 5a. Time-Normalize
        if rep_segment is None or rep_segment.shape[0] < 2: repetition_results.append( (i+1, "Error: Invalid Segment", float('inf')) ); continue
        time_norm_rep_segment = time_normalize_sequence(rep_segment, target_rep_length)
        if np.isnan(time_norm_rep_segment).all(): repetition_results.append( (i+1, "Error: Time Norm Failed", float('inf')) ); continue
        time_norm_rep_segment = np.nan_to_num(time_norm_rep_segment, nan=0.0)

        # 5b. Compare rep against baselines matching the comparison_angle_key
        min_rep_distance, best_rep_label_for_this_rep = float('inf'), None
        for exercise_label, angle_dict in averaged_baseline_data.items():
            baseline_to_compare = angle_dict.get(comparison_angle_key)
            if baseline_to_compare is None or time_norm_rep_segment.shape != baseline_to_compare.shape: continue
            safe_baseline = np.nan_to_num(baseline_to_compare, nan=0.0)
            try: distance, _ = fastdtw(time_norm_rep_segment, safe_baseline, dist=euclidean)
            except Exception: continue
            if distance < min_rep_distance: min_rep_distance, best_rep_label_for_this_rep = distance, exercise_label

        # 5c. Apply threshold specific to the matched exercise and comparison_angle_key
        matched_label_for_rep = "No Match / Inconclusive"
        threshold_to_use = fallback_threshold
        if best_rep_label_for_this_rep:
            threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get(comparison_angle_key)
            if threshold_specific is None and comparison_angle_key != 'overall':
                 threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get('overall')
            if threshold_specific is not None: threshold_to_use = threshold_specific
            if min_rep_distance < threshold_to_use: matched_label_for_rep = best_rep_label_for_this_rep
            else: matched_label_for_rep = f"No Match (Closest: {best_rep_label_for_this_rep}, Dist: {min_rep_distance:.2f} >= {threshold_to_use:.2f})"
        repetition_results.append( (i+1, matched_label_for_rep, min_rep_distance) )

    # Step 6: Return Results
    end_time = time.time()
    print(f"--- Recognition finished in {end_time - start_time:.2f}s ---")
    return repetition_results, num_detected_reps

print("\n === All Core Processing Functions Defined (Config-Driven, Angle-Aware) === ")


 === All Core Processing Functions Defined (Config-Driven, Angle-Aware) === 


In [252]:
# === Cell 5.7: Baseline Loading Function ===

In [253]:
# === Cell 5.7: Baseline Loading Function (Updated for Overall Average) ===
# === Cell 5.7: Baseline Loading Function (Using Angle from Filename) ===
import re # Make sure re is imported

def load_baseline_data(baseline_root_dir, hrnet_model, joint_names, target_rep_length, exercise_config,
                       prominence_range, distance_range, # These 7 are positional
                       threshold_std_multiplier=DEFAULT_THRESHOLD_STD_MULTIPLIER, # Keyword 1
                       min_reps_for_threshold=DEFAULT_MIN_REPS_FOR_THRESHOLD    # Keyword 2
                       ):
    """
    Loads baselines, parses angle from filename, tunes params per angle,
    averages per angle AND overall, calculates thresholds per angle AND overall.
    Returns nested dicts: averaged_baselines, tuned_counting_params, exercise_specific_thresholds
    """
    # Initialize nested dictionaries
    averaged_baseline_data = {} # Structure: {exercise: {angle: avg_sequence}}
    tuned_counting_params = {}  # Structure: {exercise: {angle: (p, d)}}
    exercise_specific_thresholds = {} # Structure: {exercise: {angle: threshold}}
    start_time_total = time.time()

    print(f"Loading Baselines (using angle from filename)... from: {baseline_root_dir}")
    if not os.path.isdir(baseline_root_dir): return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds

    for exercise_label in sorted(os.listdir(baseline_root_dir)):
        exercise_folder_path = os.path.join(baseline_root_dir, exercise_label)
        if not os.path.isdir(exercise_folder_path): continue
        print(f"\nProcessing baseline exercise: {exercise_label}")

        # Initialize storage for this exercise (per angle)
        # Use specific angle keys + 'unknown' + 'overall' later
        all_reps_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []}
        labeled_data_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []}
        # Initialize result dicts for this exercise
        averaged_baseline_data[exercise_label] = {}
        tuned_counting_params[exercise_label] = {}
        exercise_specific_thresholds[exercise_label] = {}

        video_files = sorted([f for f in os.listdir(exercise_folder_path) if f.lower().endswith(('.mp4', '.avi', '.mov'))])
        print(f"  Found {len(video_files)} video file(s).")

        # --- Loop 1: Process videos, parse angle/reps, segment, normalize time, store reps by PARSED angle ---
        for video_file in video_files:
            video_path = os.path.join(exercise_folder_path, video_file)

            # --- Parse True Rep Count ---
            true_reps = None
            rep_match = re.search(r'_(\d+)reps', video_file, re.IGNORECASE)
            if rep_match: true_reps = int(rep_match.group(1))

            # --- Parse Angle ---
            parsed_angle = 'unknown' # Default if not found
            # Look for _left_, _right_, _front_ patterns (case-insensitive)
            angle_match = re.search(r'_(left|right|front)', video_file, re.IGNORECASE)
            if angle_match:
                parsed_angle = angle_match.group(1).lower()
            # else: print(f"    Warning: Angle (left/right/front) not found in filename: {video_file}. Treating as 'unknown'.")

            print(f"    Video: {video_file}, True Reps: {true_reps}, Parsed Angle: {parsed_angle}")

            # --- Process Video ---
            raw_seq = process_single_video(video_path, hrnet_model, EXPECTED_JOINT_COUNT)
            if raw_seq is None or raw_seq.shape[0] == 0: continue
            spatially_norm_seq = normalize_pose_sequence(raw_seq, joint_names)
            if spatially_norm_seq is None or np.all(np.isnan(spatially_norm_seq)): continue
            cleaned_seq = handle_nan_values(spatially_norm_seq)
            if cleaned_seq.shape[0] < 2: continue

            # Store cleaned sequence and true reps for tuning (by parsed angle)
            if true_reps is not None:
                labeled_data_by_angle[parsed_angle].append((cleaned_seq, true_reps))

            # Segment using default params
            repetitions = segment_repetitions(cleaned_seq, exercise_label, joint_names, current_exercise_config_dict)
            if not repetitions: continue

            # Time normalize and store reps by parsed angle
            for rep_segment in repetitions:
                if rep_segment is not None and rep_segment.shape[0] >= 2:
                    time_norm_rep = time_normalize_sequence(rep_segment, target_rep_length)
                    if not np.isnan(time_norm_rep).all():
                        all_reps_by_angle[parsed_angle].append(time_norm_rep)
        # --- End Video Loop ---

        # --- Loop 2: Process per detected angle category ---
        all_reps_across_angles = [] # Collect all valid reps for overall average
        valid_angles_processed = []

        # Iterate through specific angles found + 'unknown'
        for angle in ['left', 'right', 'front', 'unknown']:
            labeled_data_for_this_angle = labeled_data_by_angle[angle]
            normalized_reps_for_this_angle = all_reps_by_angle[angle]

            if not normalized_reps_for_this_angle:
                 # print(f"  No valid repetitions found for angle category: '{angle}'.") # Verbose
                 continue # Skip tuning, averaging, threshold for this angle

            print(f"\n  Processing angle category: '{angle}' for {exercise_label}")
            valid_angles_processed.append(angle)
            all_reps_across_angles.extend(normalized_reps_for_this_angle) # Add to overall list

            # Tune parameters for this angle
            if labeled_data_for_this_angle:
                tuned_p, tuned_d = tune_counting_parameters(exercise_label, labeled_data_for_this_angle, joint_names, prominence_range, distance_range)
                tuned_counting_params[exercise_label][angle] = (tuned_p, tuned_d)
            else: tuned_counting_params[exercise_label][angle] = (DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE)

            # Average sequences for this angle
            reps_stack = np.stack(normalized_reps_for_this_angle, axis=0)
            averaged_sequence_angle = np.nanmean(reps_stack, axis=0)
            averaged_baseline_data[exercise_label][angle] = np.nan_to_num(averaged_sequence_angle, nan=0.0)
            print(f"    Stored averaged baseline for angle '{angle}'.")

            # Derive threshold for this angle
            derived_threshold_angle = None
            if len(normalized_reps_for_this_angle) >= min_reps_for_threshold:
                intra_distances_angle = []
                safe_avg_angle = np.nan_to_num(averaged_sequence_angle, nan=0.0)
                for norm_rep in normalized_reps_for_this_angle:
                     safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                     if safe_rep.shape == safe_avg_angle.shape:
                         try: dist, _ = fastdtw(safe_rep, safe_avg_angle, dist=euclidean); intra_distances_angle.append(dist)
                         except Exception: pass
                if len(intra_distances_angle) >= min_reps_for_threshold:
                     mean_dist, std_dist = np.mean(intra_distances_angle), np.std(intra_distances_angle)
                     derived_threshold_angle = mean_dist + threshold_std_multiplier * std_dist
                     print(f"    Derived Threshold for angle '{angle}': {derived_threshold_angle:.2f}")
            exercise_specific_thresholds[exercise_label][angle] = derived_threshold_angle
        # --- End Angle Loop ---

        # --- Calculate and Store Overall Average and Threshold ---
        print(f"\n  Processing 'overall' category for {exercise_label}")
        if len(all_reps_across_angles) >= min_reps_for_threshold:
             print(f"    Averaging {len(all_reps_across_angles)} total repetitions for 'overall' baseline.")
             overall_reps_stack = np.stack(all_reps_across_angles, axis=0)
             overall_averaged_sequence = np.nanmean(overall_reps_stack, axis=0)
             averaged_baseline_data[exercise_label]['overall'] = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             print(f"    Stored 'overall' averaged baseline.")

             # Derive overall threshold
             overall_intra_distances = []
             safe_overall_avg = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             for norm_rep in all_reps_across_angles:
                  safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                  if safe_rep.shape == safe_overall_avg.shape:
                       try: dist, _ = fastdtw(safe_rep, safe_overall_avg, dist=euclidean); overall_intra_distances.append(dist)
                       except Exception: pass
             if len(overall_intra_distances) >= min_reps_for_threshold:
                  mean_dist, std_dist = np.mean(overall_intra_distances), np.std(overall_intra_distances)
                  derived_threshold_overall = mean_dist + threshold_std_multiplier * std_dist
                  exercise_specific_thresholds[exercise_label]['overall'] = derived_threshold_overall
                  print(f"    Derived 'Overall' Threshold: {derived_threshold_overall:.2f}")
             else: exercise_specific_thresholds[exercise_label]['overall'] = None
        else:
             print(f"    Warning: Not enough total valid reps ({len(all_reps_across_angles)}) to create 'overall' baseline/threshold.")
             averaged_baseline_data[exercise_label]['overall'] = None
             exercise_specific_thresholds[exercise_label]['overall'] = None
        # --- End Overall Calculation ---

    # --- Function Return ---
    end_time_total = time.time()
    # ... (print completion messages) ...
    return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds


# ... (Rest of Cell 5 functions: _extract_trajectory_and_find_peaks, segment_repetitions, etc.) ...
# !!! Important: The 'calculate_exercise_trajectory' function is no longer needed with this structure !!!
# !!! You can REMOVE the definition of 'calculate_exercise_trajectory' from Cell 5.3 !!!
# Ensure segment_repetitions, count_repetitions, tune_counting_parameters now use EXERCISE_CONFIG

# Need to update segment_repetitions, count_repetitions, tune_counting_parameters
# to fetch logic from EXERCISE_CONFIG instead of having internal if/elifs

print("Baseline loading function updated (load_baseline_data uses angle from filename).")
# print("REMOVE calculate_exercise_trajectory. UPDATE segment_repetitions, count_repetitions, tune_counting_parameters TO USE EXERCISE_CONFIG.")

Baseline loading function updated (load_baseline_data uses angle from filename).


In [254]:
# === Cell 5.8: Recognition Function ===

In [255]:
# === Cell 5.8: Recognition Function (Updated for Angle-Specific Comparison) ===

def recognize_exercise(new_video_path, averaged_baseline_data, hrnet_model, joint_names,
                       target_rep_length, tuned_counting_params, exercise_specific_thresholds, # Added thresholds dict
                       fallback_threshold=DTW_DISTANCE_THRESHOLD): # Global fallback
    """
    Recognizes exercise using angle detection, angle-specific (or overall) baselines,
    angle-specific (or overall/fallback) thresholds, and tuned segmentation parameters.
    """
    print(f"\n--- Recognizing Exercise (Angle-Aware Per Rep) for: {os.path.basename(new_video_path)} ---")
    start_time = time.time()
    if tuned_counting_params is None: tuned_counting_params = {}
    if exercise_specific_thresholds is None: exercise_specific_thresholds = {}
    default_p, default_d = DEFAULT_PEAK_PROMINENCE, DEFAULT_PEAK_DISTANCE

    # --- Step 1: Process Video -> Spatial Norm -> Clean ---
    new_raw_seq = process_single_video(new_video_path, hrnet_model, EXPECTED_JOINT_COUNT)
    if new_raw_seq is None or new_raw_seq.shape[0] == 0: return [], 0
    new_spatially_norm_seq = normalize_pose_sequence(new_raw_seq, joint_names)
    if new_spatially_norm_seq is None or np.all(np.isnan(new_spatially_norm_seq)): return [], 0
    new_sequence_clean = handle_nan_values(new_spatially_norm_seq)
    if new_sequence_clean.shape[0] < 2: return [], 0

    # --- Step 2: Detect Angle of Test Video ---
    test_video_angle = detect_camera_angle(new_sequence_clean, joint_names)
    print(f"  Detected test video angle: '{test_video_angle}'")

    # --- Step 3: Initial Guess & Parameter Selection for Segmentation ---
    # (This part remains similar, but we store the initial guess label)
    new_sequence_time_norm = time_normalize_sequence(new_sequence_clean, target_rep_length) # For initial guess
    if np.isnan(new_sequence_time_norm).all(): return [], 0 # Handle failure
    new_sequence_time_norm = np.nan_to_num(new_sequence_time_norm, nan=0.0)

    initial_min_distance, initial_best_label = float('inf'), None
    segment_p, segment_d = default_p, default_d
    segment_label_guess = 'unknown'

    if not averaged_baseline_data: print("E: No baseline data."); return [], 0
    print("  Performing initial comparison for segmentation parameter selection...")
    # Compare against FRONT view baselines for initial guess? Or OVERALL? Let's use overall if available.
    guess_angle_key = 'overall' if any('overall' in d for d in averaged_baseline_data.values() if isinstance(d, dict)) else 'front' # Prefer overall if exists

    for ex_label, angle_dict in averaged_baseline_data.items():
        avg_base_seq = angle_dict.get(guess_angle_key) # Get overall or front baseline
        if avg_base_seq is None or new_sequence_time_norm.shape != avg_base_seq.shape: continue
        safe_base = np.nan_to_num(avg_base_seq, nan=0.0)
        try: dist, _ = fastdtw(new_sequence_time_norm, safe_base, dist=euclidean)
        except Exception: continue
        if dist < initial_min_distance: initial_min_distance, initial_best_label = dist, ex_label

    # Select parameters based on initial guess AND detected angle
    # Use tuned params specific to the detected angle of the test video if known
    angle_to_tune_with = test_video_angle if test_video_angle != 'unknown' else 'front' # Use front if unknown? or overall? Needs decision. Let's try front for now.

    if initial_best_label:
        segment_label_guess = initial_best_label # Use label for trajectory logic
        # Get params tuned for the specific detected angle (or fallback)
        tuned_p, tuned_d = tuned_counting_params.get(initial_best_label, {}).get(angle_to_tune_with, (default_p, default_d))
        segment_p, segment_d = tuned_p, tuned_d
        print(f"  Initial guess: {initial_best_label}. Using its '{angle_to_tune_with}' tuned params for segmentation (P={segment_p:.3f}, D={segment_d}).")
    else:
        print(f"  Initial guess failed. Using default params for segmentation.")
        # segment_label_guess remains 'unknown'

    # --- Step 4: Segment Test Video ---
    test_repetitions = segment_repetitions(new_sequence_clean, segment_label_guess, joint_names,
                                           peak_prominence=segment_p, peak_distance=segment_d)
    num_detected_reps = len(test_repetitions)
    print(f"  Detected {num_detected_reps} potential repetitions in test video.")
    if num_detected_reps == 0: return [], 0

    # --- Step 5: Compare Each Repetition Segment (Angle-Aware) ---
    repetition_results = []
    print(f"  Comparing each detected repetition against '{test_video_angle if test_video_angle != 'unknown' else 'overall'}' baselines...")

    # Determine which baseline/threshold key to use based on detected angle
    comparison_angle_key = test_video_angle if test_video_angle != 'unknown' else 'overall'

    for i, rep_segment in enumerate(test_repetitions):
        # 5a. Time-Normalize
        if rep_segment is None or rep_segment.shape[0] < 2:
             repetition_results.append( (i+1, "Error: Invalid Segment", float('inf')) ); continue
        time_norm_rep_segment = time_normalize_sequence(rep_segment, target_rep_length)
        if np.isnan(time_norm_rep_segment).all():
            repetition_results.append( (i+1, "Error: Time Norm Failed", float('inf')) ); continue
        time_norm_rep_segment = np.nan_to_num(time_norm_rep_segment, nan=0.0)

        # 5b. Compare rep against baselines matching the detected angle (or overall)
        min_rep_distance = float('inf')
        best_rep_label_for_this_rep = None

        for exercise_label, angle_dict in averaged_baseline_data.items():
            # Get the specific baseline for the determined angle key ('left','right','front', or 'overall')
            baseline_to_compare = angle_dict.get(comparison_angle_key)

            # Skip if no baseline exists for this exercise/angle combination
            if baseline_to_compare is None or time_norm_rep_segment.shape != baseline_to_compare.shape:
                 # print(f"DEBUG: No baseline for {exercise_label} angle '{comparison_angle_key}' or shape mismatch.")
                 continue

            safe_baseline = np.nan_to_num(baseline_to_compare, nan=0.0)
            try:
                distance, _ = fastdtw(time_norm_rep_segment, safe_baseline, dist=euclidean)
                if distance < min_rep_distance:
                    min_rep_distance = distance
                    best_rep_label_for_this_rep = exercise_label
            except Exception as e:
                 print(f"      Warn: DTW Error Rep {i+1} vs {exercise_label}/{comparison_angle_key}: {e}")
                 continue

        # 5c. Apply threshold (specific to the matched exercise and angle/overall)
        matched_label_for_rep = "No Match / Inconclusive" # Default
        threshold_to_use = fallback_threshold # Start with global fallback

        if best_rep_label_for_this_rep: # If a closest match was found
             # Get the specific threshold for the matched exercise and comparison angle key
             threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get(comparison_angle_key)
             if threshold_specific is not None:
                  threshold_to_use = threshold_specific
             # else: print(f"Debug: Using fallback threshold for {best_rep_label_for_this_rep}/{comparison_angle_key}") # Optional

             # Now compare distance to the selected threshold
             if min_rep_distance < threshold_to_use:
                  matched_label_for_rep = best_rep_label_for_this_rep
             else:
                  # Update label to show why it failed threshold
                  matched_label_for_rep = f"No Match (Closest: {best_rep_label_for_this_rep}, Dist: {min_rep_distance:.2f} >= {threshold_to_use:.2f})"


        repetition_results.append( (i+1, matched_label_for_rep, min_rep_distance) )

    # --- Step 6: Return Results ---
    end_time = time.time()
    print(f"--- Recognition finished in {end_time - start_time:.2f}s ---")
    return repetition_results, num_detected_reps

print("Recognition function updated (recognize_exercise supports angle-specific comparison).")

Recognition function updated (recognize_exercise supports angle-specific comparison).


In [256]:

import os
import numpy as np

print(" Running Configuration PreCheck ")
config_ok = True
error_messages = []

if 'MODEL_PATH' not in locals() or not MODEL_PATH:
    error_messages.append("❌ MODEL_PATH: Not defined.")
    config_ok = False
elif not os.path.exists(MODEL_PATH):
    error_messages.append(f"❌ MODEL_PATH: File not found at '{MODEL_PATH}'.")
    config_ok = False
else:
    print(f"✅ MODEL_PATH: Found ('{os.path.basename(MODEL_PATH)}').")

if 'BASELINE_ROOT_DIR' not in locals() or not BASELINE_ROOT_DIR:
    error_messages.append("❌ BASELINE_ROOT_DIR: Not defined.")
    config_ok = False
elif not os.path.isdir(BASELINE_ROOT_DIR):
    error_messages.append(f"❌ BASELINE_ROOT_DIR: Directory not found at '{BASELINE_ROOT_DIR}'. Please create it or correct the path.")
    config_ok = False
else:
    print(f"✅ BASELINE_ROOT_DIR: Found ('{BASELINE_ROOT_DIR}').")

if 'VIDEO_TO_RECOGNIZE_PATH' not in locals() or not VIDEO_TO_RECOGNIZE_PATH:
    error_messages.append("❌ VIDEO_TO_RECOGNIZE_PATH: Not defined.")
    config_ok = False
elif not os.path.exists(VIDEO_TO_RECOGNIZE_PATH):
    error_messages.append(f"❌ VIDEO_TO_RECOGNIZE_PATH: File not found at '{VIDEO_TO_RECOGNIZE_PATH}'.")
    config_ok = False
else:
    print(f"✅ VIDEO_TO_RECOGNIZE_PATH: Found ('{os.path.basename(VIDEO_TO_RECOGNIZE_PATH)}').")

if 'DTW_DISTANCE_THRESHOLD' not in locals() or not isinstance(DTW_DISTANCE_THRESHOLD, (int, float, np.number)) or np.isnan(DTW_DISTANCE_THRESHOLD):
    error_messages.append("❌ DTW_DISTANCE_THRESHOLD: Not defined or not a valid number.")
    config_ok = False
else:
    print(f"✅ DTW_DISTANCE_THRESHOLD: Set ({DTW_DISTANCE_THRESHOLD}).")

if 'TARGET_REP_LENGTH' not in locals() or not isinstance(TARGET_REP_LENGTH, int) or TARGET_REP_LENGTH <= 0:
    error_messages.append("❌ TARGET_REP_LENGTH: Not defined or not a positive integer.")
    config_ok = False
else:
    print(f"✅ TARGET_REP_LENGTH: Set ({TARGET_REP_LENGTH}).")

if 'EXPECTED_JOINT_COUNT' not in locals() or not isinstance(EXPECTED_JOINT_COUNT, int) or EXPECTED_JOINT_COUNT <= 0:
    error_messages.append("❌ EXPECTED_JOINT_COUNT: Not defined or not a positive integer.")
    config_ok = False
else:
    print(f"✅ EXPECTED_JOINT_COUNT: Set ({EXPECTED_JOINT_COUNT}).")
    if 'JOINT_NAMES_LIST' not in locals() or not isinstance(JOINT_NAMES_LIST, list):
        error_messages.append("❌ JOINT_NAMES_LIST: Not defined or not a list.")
        config_ok = False
    elif len(JOINT_NAMES_LIST) != EXPECTED_JOINT_COUNT:
        error_messages.append(f"❌ JOINT_NAMES_LIST: Length ({len(JOINT_NAMES_LIST)}) does not match EXPECTED_JOINT_COUNT ({EXPECTED_JOINT_COUNT}).")
        config_ok = False
    else:
        print(f"✅ JOINT_NAMES_LIST: Defined with correct length ({len(JOINT_NAMES_LIST)}).")
if 'HRNET_MODEL' not in locals() or HRNET_MODEL is None:
     error_messages.append("❌ HRNET_MODEL: Model was not loaded successfully in Cell 3.")
     config_ok = False
else:
     print("✅ HRNET_MODEL: Loaded.")


 Running Configuration PreCheck 
✅ MODEL_PATH: Found ('pose_hrnet_w48_256x192.pth').
✅ BASELINE_ROOT_DIR: Found ('C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/baselines2').
✅ VIDEO_TO_RECOGNIZE_PATH: Found ('test_excercise.mp4').
✅ DTW_DISTANCE_THRESHOLD: Set (500.0).
✅ TARGET_REP_LENGTH: Set (100).
✅ EXPECTED_JOINT_COUNT: Set (17).
✅ JOINT_NAMES_LIST: Defined with correct length (17).
✅ HRNET_MODEL: Loaded.


In [257]:
if 'HRNET_MODEL' not in locals() or HRNET_MODEL is None:
    print("HRNet Model not loaded. Please run Cell 3 successfully first.")
elif 'config_ok' not in locals() or not config_ok:
    print("Configuration check in Cell 5.5 failed. Please fix issues before running baseline loading.")

else:
    print("=================================================================")
    print(" Step 1: Loading Baselines, Tuning Params, Deriving Thresholds ")
    print("=================================================================")

    # Reset variables before loading (important for re-running the cell)
    averaged_baselines = {}
    tuned_params = {}
    derived_thresholds = {}

    # --- Load Averaged Baseline Data, Tuned Params, AND Derived Thresholds ---
    # Calls the latest function defined in Cell 5
    averaged_baselines, tuned_params, derived_thresholds = load_baseline_data(
        BASELINE_ROOT_DIR,          # Positional 1
        HRNET_MODEL,                # Positional 2
        JOINT_NAMES_LIST,           # Positional 3
        TARGET_REP_LENGTH,          # Positional 4
        EXERCISE_CONFIG,            # Positional 5
        TUNING_PROMINENCE_RANGE,    # Positional 6
        TUNING_DISTANCE_RANGE,      # Positional 7
        threshold_std_multiplier=DEFAULT_THRESHOLD_STD_MULTIPLIER, # Keyword 1
        min_reps_for_threshold=DEFAULT_MIN_REPS_FOR_THRESHOLD   # Keyword 2
    )

    # Check results
    if averaged_baselines: # Check if primary result is non-empty
        print("\n--- Baseline Data Loaded/Processed Successfully --- 👍")
        # Add more checks if needed for tuned_params and derived_thresholds
    else:
        print("\n--- Baseline Data Loading/Processing Failed --- 👎")

 Step 1: Loading Baselines, Tuning Params, Deriving Thresholds 
Loading Baselines (using angle from filename)... from: C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/baselines2

Processing baseline exercise: tricep Pushdown
  Found 4 video file(s).
    Video: tricep_pushdown_3reps_front_video1.mp4, True Reps: 3, Parsed Angle: front


NameError: name 'current_exercise_config_dict' is not defined

In [ ]:
# === Cell 6.5: Display Derived Thresholds (Optional) ===

import pprint # For potentially prettier dictionary printing

print("\n--- Derived Recognition Thresholds (Per Angle + Overall) ---")
if 'derived_thresholds' in locals() and derived_thresholds:
    pprint.pprint(derived_thresholds)
    print("----------------------------------------------------------")
elif 'derived_thresholds' in locals():
     print("  No thresholds were derived (dictionary is empty).")
else:
    print("  Derived thresholds dictionary not found. Did Cell 6 run successfully?")

In [ ]:
DTW_DISTANCE_THRESHOLD = 7000

In [ ]:
# Set to 'left', 'right', 'front', or leave as None to use 'overall' baselines/thresholds
manual_angle_hint = None # Or 'left', 'right', 'front'


video_to_test = VIDEO_TO_RECOGNIZE_PATH 
proceed_to_recognition = True
if 'averaged_baselines' not in locals() or not averaged_baselines \
   or 'tuned_params' not in locals() or tuned_params is None \
   or 'derived_thresholds' not in locals() or derived_thresholds is None: # Added check for thresholds
    error_messages.append("❌ Error: Baseline data, tuned parameters, or derived thresholds not found.")
    proceed_to_recognition = False
# ... (other checks) ...
elif not os.path.exists(video_to_test):
     error_messages.append(f"❌ Error: Test video file not found at '{video_to_test}'. Please check the path.")
     proceed_to_recognition = False

# --- Only proceed if all checks passed ---
if proceed_to_recognition:
    # ... (print header) ...

    # --- Run Recognition (Pass derived_thresholds) ---
    list_of_rep_results, total_reps_detected = recognize_exercise(
        video_to_test,
        averaged_baselines,
        HRNET_MODEL,
        JOINT_NAMES_LIST,
        DTW_DISTANCE_THRESHOLD, # This is now mainly a fallback
        TARGET_REP_LENGTH,
        tuned_params,
        derived_thresholds # Pass the derived thresholds here
    )
if proceed_to_recognition:
    print("==============================================")
    print(f"    Step 2: Recognizing Exercise in Video (Per Rep) ")
    print(f"    Video: {os.path.basename(video_to_test)} ")
    if manual_angle_hint: print(f"    Angle Hint: '{manual_angle_hint}'")
    print("==============================================")

    # --- Run Recognition (Pass hint_angle and config) ---
    list_of_rep_results, total_reps_detected = recognize_exercise(
        new_video_path=video_to_test,
        averaged_baseline_data=averaged_baselines,
        hrnet_model=HRNET_MODEL,
        joint_names=JOINT_NAMES_LIST,
        exercise_config=EXERCISE_CONFIG, # Pass the config
        target_rep_length=TARGET_REP_LENGTH,
        tuned_counting_params=tuned_params,
        exercise_specific_thresholds=derived_thresholds,
        hint_angle=manual_angle_hint, # Pass the hint
        fallback_threshold=DTW_DISTANCE_THRESHOLD # Pass fallback
    )

    # --- Print Results ---
    print("\n==============================================")
    print(f"    FINAL RESULT for {os.path.basename(video_to_test)}")
    print("==============================================")
    print(f"  Detected {total_reps_detected} Repetitions.") # Uses count from segmentation step

    if not list_of_rep_results:
        print("  No valid repetition results to display (segmentation might have failed or video was problematic).")
    else:
        print("\n  --- Per-Repetition Analysis ---")
        match_counts = {}
        successful_reps = 0
        for rep_index, matched_label, min_distance in list_of_rep_results:
            status = ""
            # Check for specific error strings first
            if isinstance(matched_label, str) and matched_label.startswith("Error"):
                 status = f"-> {matched_label}"
            # Check for inconclusive match (below threshold check is now inside recognize_exercise)
            elif isinstance(matched_label, str) and matched_label.startswith("No Match"):
                 status = f"-> {matched_label}" # Includes threshold info now
            # Otherwise, it's a successful match for this rep
            else:
                status = f"-> Match: {matched_label} (Dist: {min_distance:.2f})"
                match_counts[matched_label] = match_counts.get(matched_label, 0) + 1
                successful_reps += 1
            print(f"    Rep {rep_index}: {status}")

        print("\n  --- Overall Summary ---")
        if successful_reps > 0:
             if match_counts:
                 # Find the exercise label with the highest count
                 majority_label = max(match_counts, key=match_counts.get)
                 majority_count = match_counts[majority_label]
                 print(f"  Overall Predominant Match: {majority_label} ({majority_count}/{successful_reps} successfully matched reps)")
             else:
                 # This case should not happen if successful_reps > 0, but included for safety
                  print("  No specific exercises were matched consistently.")
             # Use total_reps_detected which comes from segmentation
             print(f"  Total Reps Successfully Matched (below threshold): {successful_reps} / {total_reps_detected}")
        elif total_reps_detected > 0: # Reps detected but none matched
             print("  No repetitions were successfully matched to any baseline exercise.")
        else: # No reps detected at all
             print("  No repetitions were detected in the video to analyze.")
